# Import Essential Modules

# Implementations

## o3 Draft

In [ ]:
#!/usr/bin/env python3
"""
video_chain_vertex_ai.py
Industrial-grade pipeline for chained video generation using
Google Vertex-AI VEO-2.  © MIT CSAIL 2024
"""

from __future__ import annotations

# ──────────────────────────────────────────────────────────
# Standard library
import os
import io
import json
import shutil
import logging
from contextlib import contextmanager
from pathlib import Path
from typing import List, Generator, Tuple

# ──────────────────────────────────────────────────────────
# Third-party
import cv2  # OpenCV-Python
import numpy as np
from moviepy.editor import VideoFileClip, concatenate_videoclips  # type: ignore

from google.api_core.exceptions import GoogleAPICallError
from google.auth.exceptions import DefaultCredentialsError
import vertexai
from vertexai.preview.generative_models import (
    GenerativeModel,
    VideoGenerationResponse,
    Image as VertexImage,
)

# ──────────────────────────────────────────────────────────
# Logging config
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
)
LOGGER = logging.getLogger("video_chain_vertex_ai")


# ──────────────────────────────────────────────────────────
# Helper context manager for Vertex calls
@contextmanager
def vertex_error_guard(step_description: str) -> Generator[None, None, None]:
    """
    Context manager that converts Google API errors into user-friendly RuntimeError.

    Parameters
    ----------
    step_description : str
        Human-readable description of the API step, for logging and error messages.
    """
    try:
        yield
    except (GoogleAPICallError, DefaultCredentialsError) as err:
        LOGGER.error("Vertex-AI failure during %s: %s", step_description, err)
        raise RuntimeError(f"Vertex-AI failure during {step_description}") from err


# ──────────────────────────────────────────────────────────
# Main public callable
def generate_chained_videos(
    reference_image_path: str,
    prompts: List[str],
    api_key: str,
    output_folder_name: str,
    *,
    project: str | None = None,
    location: str = "us-central1",
    video_duration: int = 6,
    fps: int = 24,
) -> Path:
    """
    Generate a chain of videos from an initial still image & prompt list using
    Vertex-AI VEO-2, then concatenate them into a single final video.

    Parameters
    ----------
    reference_image_path : str
        Absolute or relative path of the seed image.
    prompts : List[str]
        Ordered list of textual scene descriptions.
    api_key : str
        Vertex-AI API key (will be injected into env var GOOGLE_API_KEY).
    output_folder_name : str
        Name of the folder (created inside user `$HOME`) that will store videos.
    project : str | None, optional
        GCP project ID.  If None, uses env var GOOGLE_CLOUD_PROJECT.
    location : str, optional
        Vertex region; default 'us-central1'.
    video_duration : int, optional
        Duration **per prompt** in seconds (uniform to ease concatenation).
    fps : int, optional
        Frames-per-second for generated videos.

    Returns
    -------
    Path
        Absolute path to the stitched final video `<out_dir>/final.mp4`.

    Raises
    ------
    FileNotFoundError
        If the reference image does not exist.
    ValueError
        For empty prompt list or invalid arguments.
    RuntimeError
        On Vertex-AI failures or OpenCV processing problems.
    """
    # ──────────────────────────────────────────────────────
    # 0. Input validation
    ref_path = Path(reference_image_path).expanduser().resolve()
    if not ref_path.exists() or not ref_path.is_file():
        raise FileNotFoundError(f"Reference image not found: {ref_path}")
    if not prompts:
        raise ValueError("Prompts list is empty.")
    if not api_key.strip():
        raise ValueError("API key is empty.")
    if not output_folder_name.strip():
        raise ValueError("Output folder name is empty.")

    # ──────────────────────────────────────────────────────
    # 1. Create output directory in $HOME
    out_dir = Path.home() / output_folder_name
    out_dir.mkdir(parents=True, exist_ok=True)
    LOGGER.info("Output directory: %s", out_dir)

    # ──────────────────────────────────────────────────────
    # 2. Read reference image
    reference_bgr = cv2.imread(str(ref_path))
    if reference_bgr is None:
        raise RuntimeError(f"OpenCV could not decode image: {ref_path}")
    # Convert last frame to PNG bytes for Vertex
    reference_png_bytes = _encode_frame_to_png(reference_bgr)

    # ──────────────────────────────────────────────────────
    # 3. Initialise Vertex-AI client (once)
    os.environ["GOOGLE_API_KEY"] = api_key
    vertexai.init(project=project, location=location)
    model = GenerativeModel("veo@002")

    # Uniform video_config dict
    video_cfg = {
        "duration": f"{video_duration}s",
        "fps": fps,
        "format": "MP4",
    }

    # Keep path list for later concatenation
    generated_paths: List[Path] = []

    # ──────────────────────────────────────────────────────
    # 4. Iterate over prompts
    last_frame_bytes = reference_png_bytes  # first iteration seed
    for idx, prompt in enumerate(prompts, start=1):
        video_mode = "GENERATE" if idx == 1 else "EDIT"
        filename = f"{idx}.mp4"
        video_path = out_dir / filename
        LOGGER.info("↳  Generating video #%d (%s) …", idx, video_mode)

        # Build request payload
        vertex_images = [VertexImage.from_bytes(last_frame_bytes)]
        request_payload = {
            "prompt": prompt,
            "images": vertex_images,
            "video_config": {**video_cfg, "video_mode": video_mode},
        }

        # Call Vertex-AI VEO-2
        with vertex_error_guard(f"video generation #{idx}"):
            response: VideoGenerationResponse = model.generate_video(**request_payload)

        # Save video bytes
        _dump_vertex_video_to_disk(response, video_path)
        generated_paths.append(video_path)

        # Decompose & grab last frame for next iteration
        last_frame_bgr = _extract_last_frame(video_path)
        last_frame_bytes = _encode_frame_to_png(last_frame_bgr)

    # ──────────────────────────────────────────────────────
    # 5. Concatenate all videos into final.mp4
    final_path = out_dir / "final.mp4"
    _concatenate_videos(generated_paths, final_path)
    LOGGER.info("✅  Final stitched video written to %s", final_path)

    # ──────────────────────────────────────────────────────
    # 6. Return path to caller for downstream usage
    return final_path


# ──────────────────────────────────────────────────────────
# ----------------------- helper utils --------------------
def _encode_frame_to_png(bgr_array: np.ndarray) -> bytes:
    """
    Convert a BGR image (OpenCV) to PNG bytes for Vertex.
    """
    success, buf = cv2.imencode(".png", bgr_array)
    if not success:
        raise RuntimeError("OpenCV failed to encode frame to PNG.")
    return buf.tobytes()


def _dump_vertex_video_to_disk(
    response: VideoGenerationResponse,
    out_path: Path,
) -> None:
    """
    Persist the video bytes contained in a Vertex response to disk.

    The response contains:
       • `response.media[0].data`  – raw bytes
       • OR `response.media[0].uri` – GCS URI
    We stream bytes for maximal portability.
    """
    media = response.media[0]
    if media.data:
        video_bytes = media.data
        out_path.write_bytes(video_bytes)
    elif media.uri:
        # Fallback: download from GCS URI
        import requests  # local import to avoid hard dep
        with requests.get(media.uri, stream=True, timeout=45) as r:
            r.raise_for_status()
            with out_path.open("wb") as fh:
                shutil.copyfileobj(r.raw, fh)
    else:
        raise RuntimeError("Vertex response contained neither data nor URI.")


def _extract_last_frame(video_path: Path) -> np.ndarray:
    """
    Open `video_path` with OpenCV and return the last frame as BGR ndarray.
    """
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise RuntimeError(f"Unable to open video: {video_path}")
    last_frame: np.ndarray | None = None
    while True:
        grabbed, frame = cap.read()
        if not grabbed:
            break
        last_frame = frame
    cap.release()
    if last_frame is None:
        raise RuntimeError(f"No frames found in video: {video_path}")
    return last_frame


def _concatenate_videos(video_paths: List[Path], out_path: Path) -> None:
    """
    Concatenate a list of video files (same FPS & resolution) preserving order.
    """
    clips = [VideoFileClip(str(p)) for p in video_paths]
    final_clip = concatenate_videoclips(clips, method="compose")
    final_clip.write_videofile(str(out_path), codec="libx264", audio=False)
    # Close resources to avoid memory leaks
    final_clip.close()
    for c in clips:
        c.close()

## 3.  Usage example

from video_chain_vertex_ai import generate_chained_videos

final_path = generate_chained_videos(
    reference_image_path="/data/seed.jpg",
    prompts=[
        "A serene sunrise over a calm ocean, cinematic.",
        "Clouds gather and a storm begins to brew, dramatic.",
        "Rainbow appears as the storm clears, hopeful."
    ],
    api_key="AIza…your-key…",
    output_folder_name="ocean_story",
    project="my-gcp-project",  # optional if ADC set
)
print("Stitched video is at:", final_path)



## Opus 4 Draft



In [ ]:
import os
import base64
import time
import logging
from pathlib import Path
from typing import List, Optional, Dict, Any, Tuple
from datetime import datetime
import json

import cv2
import numpy as np
from PIL import Image
import imageio
from moviepy.editor import VideoFileClip, concatenate_videoclips
from google.cloud import aiplatform
from google.oauth2 import service_account
import vertexai
from vertexai.preview.vision_models import Image as VertexImage, VideoGenerationModel

# Configure logging for production-grade error tracking
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


def generate_sequential_videos_with_veo2(
    reference_image_path: str,
    scene_prompts: List[str],
    vertex_api_key: str,
    output_folder_name: str
) -> Path:
    """
    Generate a sequence of videos using Google's VEO-2 model, where each video
    uses the last frame of the previous video as input.

    This function implements a sequential video generation pipeline using VEO-2,
    creating smooth transitions between scenes by using the last frame of each
    generated video as the starting point for the next.

    Args:
        reference_image_path (str): File path to the initial reference image.
            Must be a valid image file (JPEG, PNG, etc.).
        scene_prompts (List[str]): List of prompts describing each scene to generate.
            Each prompt should be descriptive and specific for best results.
        vertex_api_key (str): Google Cloud Vertex AI API key for authentication.
            Must have permissions for VEO-2 video generation.
        output_folder_name (str): Name of the folder to create in home directory
            for storing generated videos.

    Returns:
        Path: Path to the final stitched video file.

    Raises:
        ValueError: If inputs are invalid (empty prompts, invalid paths, etc.).
        FileNotFoundError: If reference image doesn't exist.
        PermissionError: If unable to create output directory.
        RuntimeError: If video generation or processing fails.

    Example:
        >>> final_video = generate_sequential_videos_with_veo2(
        ...     "reference.jpg",
        ...     ["Scene 1: Sunrise", "Scene 2: Noon", "Scene 3: Sunset"],
        ...     "your-api-key",
        ...     "generated_videos"
        ... )
    """
    # Input validation to ensure all parameters meet requirements
    if not reference_image_path or not os.path.exists(reference_image_path):
        raise FileNotFoundError(f"Reference image not found: {reference_image_path}")

    if not scene_prompts or not all(isinstance(p, str) and p.strip() for p in scene_prompts):
        raise ValueError("Scene prompts must be a non-empty list of non-empty strings")

    if not vertex_api_key or not isinstance(vertex_api_key, str):
        raise ValueError("Valid Vertex API key required")

    if not output_folder_name or not isinstance(output_folder_name, str):
        raise ValueError("Output folder name must be a non-empty string")

    # Step 1: Create output directory in home directory
    # Using pathlib for cross-platform compatibility
    home_dir = Path.home()
    output_dir = home_dir / output_folder_name

    try:
        # Create directory with parents if needed, exist_ok prevents errors if exists
        output_dir.mkdir(parents=True, exist_ok=True)
        logger.info(f"Created output directory: {output_dir}")
    except PermissionError as e:
        raise PermissionError(f"Unable to create directory {output_dir}: {e}")
    except Exception as e:
        raise RuntimeError(f"Unexpected error creating directory: {e}")

    # Initialize Vertex AI with credentials
    # Setting up authentication and project configuration
    try:
        # Initialize Vertex AI client with API key
        # Note: In production, use service account credentials file
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = vertex_api_key

        # Initialize with your project and location
        # These should be configured based on your Vertex AI setup
        PROJECT_ID = "your-project-id"  # Replace with actual project ID
        LOCATION = "us-central1"  # VEO-2 availability region

        vertexai.init(project=PROJECT_ID, location=LOCATION)
        model = VideoGenerationModel.from_pretrained("veo-2")

        logger.info("Successfully initialized Vertex AI client")
    except Exception as e:
        raise RuntimeError(f"Failed to initialize Vertex AI: {e}")

    # Step 2: Read reference image into memory
    # Using PIL for robust image handling
    try:
        # Open and validate image
        reference_image = Image.open(reference_image_path)

        # Convert to RGB if necessary (handles RGBA, grayscale, etc.)
        if reference_image.mode != 'RGB':
            reference_image = reference_image.convert('RGB')

        # Validate image dimensions for VEO-2 (recommended max 1024x1024)
        max_dimension = 1024
        if max(reference_image.size) > max_dimension:
            # Resize maintaining aspect ratio
            reference_image.thumbnail((max_dimension, max_dimension), Image.Resampling.LANCZOS)
            logger.info(f"Resized image to {reference_image.size}")

        logger.info(f"Successfully loaded reference image: {reference_image.size}")

    except Exception as e:
        raise RuntimeError(f"Failed to load reference image: {e}")

    # List to store paths of all generated videos for final stitching
    generated_video_paths: List[Path] = []

    # Variable to track the current input image/frame for video generation
    current_input_image = reference_image

    # Step 3-5: Iterate through prompts and generate videos
    for iteration_index, prompt in enumerate(scene_prompts):
        try:
            # Log current iteration for debugging
            logger.info(f"Processing iteration {iteration_index + 1}/{len(scene_prompts)}: {prompt[:50]}...")

            # Convert current image to format suitable for VEO-2 API
            # VEO-2 expects base64-encoded image data
            image_buffer = io.BytesIO()
            current_input_image.save(image_buffer, format='PNG')
            image_bytes = image_buffer.getvalue()

            # Create VertexImage object for API
            vertex_image = VertexImage(image_bytes)

            # Generate video using VEO-2 with proper parameters
            # Following VEO-2 API specifications from documentation
            generation_params = {
                "prompt": prompt,
                "image": vertex_image,
                "duration": 5,  # 5 seconds per video segment
                "fps": 30,  # 30 frames per second for smooth playback
                "resolution": "720p",  # Balance quality and processing time
                "aspect_ratio": "16:9",  # Standard video aspect ratio
            }

            # Call VEO-2 model with retry logic for robustness
            max_retries = 3
            retry_delay = 5  # seconds

            for retry in range(max_retries):
                try:
                    # Generate video using VEO-2
                    logger.info(f"Calling VEO-2 API (attempt {retry + 1}/{max_retries})")

                    video_response = model.generate_video(
                        **generation_params
                    )

                    # Wait for video generation to complete
                    # VEO-2 operations are asynchronous
                    while not video_response.is_done():
                        time.sleep(2)
                        logger.info("Waiting for video generation to complete...")

                    # Retrieve generated video
                    generated_video = video_response.result()
                    break

                except Exception as e:
                    if retry < max_retries - 1:
                        logger.warning(f"VEO-2 API call failed, retrying in {retry_delay}s: {e}")
                        time.sleep(retry_delay)
                    else:
                        raise RuntimeError(f"VEO-2 API call failed after {max_retries} attempts: {e}")

            # Save generated video with sequential naming
            # Using 1-based indexing as specified
            video_filename = f"{iteration_index + 1}.mp4"
            video_path = output_dir / video_filename

            # Write video data to file
            with open(video_path, 'wb') as video_file:
                video_file.write(generated_video.video_data)

            logger.info(f"Saved video: {video_path}")
            generated_video_paths.append(video_path)

            # Decompose video into frames to extract last frame
            # Using OpenCV for efficient frame extraction
            cap = cv2.VideoCapture(str(video_path))

            if not cap.isOpened():
                raise RuntimeError(f"Failed to open video: {video_path}")

            # Get total frame count for seeking to last frame
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

            if total_frames == 0:
                raise RuntimeError(f"Video has no frames: {video_path}")

            # Seek to last frame efficiently
            cap.set(cv2.CAP_PROP_POS_FRAMES, total_frames - 1)

            # Read the last frame
            ret, last_frame = cap.read()
            cap.release()

            if not ret or last_frame is None:
                raise RuntimeError(f"Failed to extract last frame from video: {video_path}")

            # Convert OpenCV BGR format to RGB for PIL
            last_frame_rgb = cv2.cvtColor(last_frame, cv2.COLOR_BGR2RGB)

            # Convert to PIL Image for next iteration
            current_input_image = Image.fromarray(last_frame_rgb)

            logger.info(f"Extracted last frame for next iteration: {current_input_image.size}")

        except Exception as e:
            logger.error(f"Error in iteration {iteration_index + 1}: {e}")
            raise RuntimeError(f"Video generation failed at iteration {iteration_index + 1}: {e}")

    # Step 6: Stitch all videos together into final video
    try:
        logger.info(f"Stitching {len(generated_video_paths)} videos together...")

        # Load all video clips using moviepy
        # MoviePy handles codec compatibility and frame rate normalization
        video_clips = []

        for video_path in generated_video_paths:
            try:
                clip = VideoFileClip(str(video_path))
                video_clips.append(clip)
                logger.info(f"Loaded clip: {video_path.name} - Duration: {clip.duration}s")
            except Exception as e:
                raise RuntimeError(f"Failed to load video clip {video_path}: {e}")

        # Concatenate all clips in sequence
        # Method='compose' ensures smooth transitions
        final_video = concatenate_videoclips(video_clips, method="compose")

        # Define output path for final video
        final_video_path = output_dir / "final.mp4"

        # Write final video with optimal encoding settings
        # codec='libx264' for wide compatibility, audio_codec for any audio tracks
        final_video.write_videofile(
            str(final_video_path),
            codec='libx264',
            audio_codec='aac',
            temp_audiofile='temp-audio.m4a',
            remove_temp=True,
            fps=30,  # Maintain consistent frame rate
            preset='medium',  # Balance between speed and compression
            ffmpeg_params=['-crf', '23']  # Quality setting (lower = better quality)
        )

        # Clean up resources
        for clip in video_clips:
            clip.close()
        final_video.close()

        logger.info(f"Successfully created final video: {final_video_path}")

        return final_video_path

    except Exception as e:
        logger.error(f"Error stitching videos: {e}")
        raise RuntimeError(f"Failed to stitch videos together: {e}")


# Additional helper function for validating VEO-2 connectivity
def validate_veo2_connection(api_key: str) -> bool:
    """
    Validate that VEO-2 API is accessible with given credentials.

    Args:
        api_key (str): Vertex AI API key

    Returns:
        bool: True if connection successful, False otherwise
    """
    try:
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = api_key
        vertexai.init()
        model = VideoGenerationModel.from_pretrained("veo-2")
        return True
    except Exception as e:
        logger.error(f"VEO-2 connection validation failed: {e}")
        return False


# Import required for BytesIO
import io


## Gemini Pro 2.5 June Draft

In [ ]:
import os
import io
import logging
from pathlib import Path
from typing import List, Optional

# Third-party libraries
# Ensure you have installed the necessary packages:
# pip install google-cloud-aiplatform pillow opencv-python numpy
import cv2
import numpy as np
import vertexai
from google.api_core import exceptions as google_exceptions
from PIL import Image, UnidentifiedImageError
from vertexai.preview.vision_models import Video, VideoGenerationModel

# Configure logging for clear output
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def generate_sequential_video_story(
    reference_image_path: str,
    prompts: List[str],
    project_id: str,
    location: str,
    output_folder_name: str
) -> Optional[str]:
    """
    Generates a sequential video story by chaining VEO-2 model calls.

    This function creates a narrative video sequence where each segment is
    generated based on the last frame of the previous segment. It begins with a
    provided reference image and a series of prompts.

    Prerequisites:
    1.  Google Cloud SDK (`gcloud`) installed and authenticated. Run the
        following command in your terminal:
        `gcloud auth application-default login`
    2.  The Vertex AI API must be enabled for the specified `project_id`.
    3.  Necessary Python packages installed: `google-cloud-aiplatform`,
        `Pillow`, `opencv-python`, `numpy`.

    Args:
        reference_image_path (str):
            The absolute or relative file path to the initial reference image
            (e.g., 'images/start.png').
        prompts (List[str]):
            A list of strings, where each string is a prompt for a video
            segment. The list must not be empty.
        project_id (str):
            The Google Cloud Project ID to use for Vertex AI API calls.
        location (str):
            The Google Cloud location/region for the Vertex AI endpoint
            (e.g., 'us-central1').
        output_folder_name (str):
            The name of the folder to be created in the user's home
            directory to store the generated video segments and the final
            stitched video.

    Returns:
        Optional[str]:
            The absolute path to the final stitched video file if successful,
            otherwise None.

    Raises:
        ValueError: If inputs are invalid (e.g., empty prompts list, bad path).
        FileNotFoundError: If the reference_image_path does not exist.
        RuntimeError: If an API call or a video processing step fails.
    """
    # --- Input Validation ---
    # Validate that the reference_image_path is a valid file.
    if not Path(reference_image_path).is_file():
        raise FileNotFoundError(
            f"Reference image not found at path: {reference_image_path}"
        )
    # Validate that the prompts list is not empty.
    if not prompts:
        raise ValueError("The 'prompts' list cannot be empty.")
    # Validate that all other string inputs are non-empty.
    if not all([project_id, location, output_folder_name]):
        raise ValueError(
            "project_id, location, and output_folder_name must be non-empty strings."
        )

    try:
        # --- Step 1: Create a sub-directory in the home-directory ---
        logging.info("Step 1: Creating output directory.")
        # Get the user's home directory in a platform-agnostic way.
        home_dir = Path.home()
        # Define the full path for the output directory.
        output_dir = home_dir / output_folder_name
        # Create the directory. exist_ok=True prevents an error if it already exists.
        output_dir.mkdir(exist_ok=True, parents=True)
        logging.info(f"Output directory created/ensured at: {output_dir}")

        # --- Initialize Vertex AI ---
        logging.info("Initializing Vertex AI with Project ID and Location.")
        # This uses Application Default Credentials (ADC) for authentication.
        vertexai.init(project=project_id, location=location)
        # Load the VEO-2 model (or a compatible video generation model).
        # The model name "imagen-video" is used here as a placeholder for the
        # specific, potentially private preview, VEO-2 model identifier.
        model = VideoGenerationModel.from_pretrained("imagen-video@002")

        # --- Step 2: Read the initial reference image into memory ---
        logging.info("Step 2: Reading and preparing the initial reference image.")
        # This variable will hold the image data (in bytes) for each generation step.
        current_frame_bytes: Optional[bytes] = None
        try:
            # Open the image file from the provided path.
            with Image.open(reference_image_path) as img:
                # Create an in-memory binary stream.
                with io.BytesIO() as byte_stream:
                    # Save the image to the stream in PNG format (lossless).
                    img.save(byte_stream, format="PNG")
                    # Get the byte value of the stream.
                    current_frame_bytes = byte_stream.getvalue()
        except UnidentifiedImageError:
            # Handle cases where the file is not a valid image.
            raise ValueError(f"File at {reference_image_path} is not a valid image.")

        # --- Steps 3, 4, 5: Iterate, Generate, Decompose, and Chain ---
        logging.info("Starting iterative video generation loop.")
        generated_video_paths = []

        for i, prompt in enumerate(prompts):
            video_number = i + 1
            logging.info(f"--- Iteration {video_number}/{len(prompts)} ---")
            logging.info(f"Prompt: '{prompt}'")

            # --- Step 4 & 5.c/f: Generate video using the model ---
            # The input for the first iteration is the reference image.
            # For subsequent iterations, it's the last frame of the previous video.
            if current_frame_bytes is None:
                # This should not be reached if logic is correct, but is a safeguard.
                raise RuntimeError("Image bytes are missing for video generation.")

            logging.info("Generating video with Vertex AI VEO-2 model...")
            try:
                # This is the core API call. We generate a short video.
                # Cost-efficiency note: shorter videos are cheaper. A 4-second
                # length is a reasonable default for animated segments.
                response: Video = model.generate(
                    prompt=prompt,
                    # The seed image provides the starting point for generation.
                    seed_image=Image.open(io.BytesIO(current_frame_bytes)),
                    # Generate a 4-second video at 24 FPS.
                    video_length_sec=4,
                    fps=24,
                )
            except google_exceptions.GoogleAPICallError as e:
                # Catch API errors (e.g., rate limits, invalid arguments).
                raise RuntimeError(
                    f"Vertex AI API call failed at iteration {video_number}: {e}"
                ) from e

            # --- Step 5.a/d: Save the generated video to the folder ---
            video_path = output_dir / f"{video_number}.mp4"
            # The API returns video data which we write to a file.
            response.save(str(video_path))
            # Keep track of the path for the final stitching step.
            generated_video_paths.append(video_path)
            logging.info(f"Saved video segment to: {video_path}")

            # --- Prepare for the next iteration (if not the last one) ---
            if video_number < len(prompts):
                logging.info("Decomposing video to get the last frame for the next iteration.")
                # --- Step 5.b/e: Decompose the generated video into image frames ---
                video_capture = None
                try:
                    # Open the video we just saved.
                    video_capture = cv2.VideoCapture(str(video_path))
                    # Get the total number of frames in the video.
                    frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
                    if frame_count == 0:
                        raise RuntimeError(f"Video file {video_path} is empty or corrupt.")

                    # --- Step 5.c/f: Use the last frame ---
                    # Set the video position to the last frame.
                    video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_count - 1)
                    # Read the last frame.
                    success, last_frame_np = video_capture.read()

                    if not success:
                        raise RuntimeError(f"Could not read the last frame from {video_path}.")

                    # Convert the last frame (NumPy array in BGR) to bytes for the next API call.
                    # Convert color from OpenCV's BGR to Pillow's RGB.
                    last_frame_rgb = cv2.cvtColor(last_frame_np, cv2.COLOR_BGR2RGB)
                    # Create a Pillow Image object from the NumPy array.
                    last_frame_pil = Image.fromarray(last_frame_rgb)

                    # Use the same in-memory stream technique to get bytes.
                    with io.BytesIO() as byte_stream:
                        last_frame_pil.save(byte_stream, format="PNG")
                        current_frame_bytes = byte_stream.getvalue()

                    logging.info("Successfully extracted last frame for the next cycle.")

                finally:
                    # CRITICAL: Always release the video capture resource.
                    if video_capture:
                        video_capture.release()

        # --- Step 6: Stitch all generated videos together ---
        logging.info("--- All segments generated. Starting final stitching process. ---")
        if not generated_video_paths:
            logging.warning("No video segments were generated. Cannot stitch.")
            return None

        # Sort the video paths numerically to ensure correct order.
        # This prevents '10.mp4' from coming before '2.mp4'.
        sorted_paths = sorted(
            generated_video_paths, key=lambda p: int(p.stem)
        )

        # Get video properties (width, height, fps) from the first clip.
        # We assume all clips have the same properties, a reasonable expectation
        # from the same generative model session.
        first_clip = cv2.VideoCapture(str(sorted_paths[0]))
        if not first_clip.isOpened():
            raise RuntimeError(f"Failed to open first video clip at {sorted_paths[0]} for stitching.")

        frame_width = int(first_clip.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(first_clip.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(first_clip.get(cv2.CAP_PROP_FPS))
        first_clip.release()

        # Define the path for the final output video.
        final_video_path = output_dir / "final.mp4"
        # Define the video codec and create the VideoWriter object.
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(str(final_video_path), fourcc, fps, (frame_width, frame_height))

        try:
            # Iterate through each video segment in the correct order.
            for video_path in sorted_paths:
                logging.info(f"Stitching {video_path}...")
                cap = cv2.VideoCapture(str(video_path))
                try:
                    # Read each frame from the segment and write it to the final video.
                    while cap.isOpened():
                        ret, frame = cap.read()
                        if not ret:
                            break
                        # Check if frame dimensions match, a safety measure.
                        if frame.shape[1] != frame_width or frame.shape[0] != frame_height:
                            logging.warning(f"Frame mismatch in {video_path}, resizing. This may impact quality.")
                            frame = cv2.resize(frame, (frame_width, frame_height))
                        video_writer.write(frame)
                finally:
                    # Ensure each video capture is released.
                    cap.release()

            logging.info(f"Successfully stitched all segments into: {final_video_path}")
            # Return the path to the final video.
            return str(final_video_path.resolve())

        finally:
            # CRITICAL: Always release the final video writer.
            video_writer.release()
            logging.info("Final video writer released.")

    except (ValueError, FileNotFoundError, RuntimeError) as e:
        # Catch our own raised errors and log them before re-raising or exiting.
        logging.error(f"A critical error occurred: {e}")
        # Depending on desired behavior, you might re-raise or just return None
        return None
    except Exception as e:
        # Catch any unexpected errors.
        logging.error(f"An unexpected error occurred: {e}", exc_info=True)
        return None


# Gemini Pro June Draft with Websearch

In [ ]:
import os
import base64
from pathlib import Path
import cv2
from moviepy.editor import VideoFileClip, concatenate_videoclips
from PIL import Image, UnidentifiedImageError
from typing import List, Tuple

# Note: The following imports are required for Google Cloud Vertex AI.
# Ensure you have authenticated your environment correctly.
# e.g., via `gcloud auth application-default login`
from google.cloud import aiplatform
from google.api_core import exceptions as google_exceptions

# --- Helper Functions ---

def _get_last_frame_as_base64(video_path: Path) -> str:
    """
    Reads a video file, extracts its very last frame, and returns it as a base64 encoded string.

    Args:
        video_path: The path to the video file.

    Returns:
        A base64 encoded string of the last frame (PNG format).

    Raises:
        IOError: If the video file cannot be opened or read.
        cv2.error: If there is an OpenCV-specific error.
    """
    # Intext Comment: Check if the video file exists before proceeding.
    if not video_path.is_file():
        raise FileNotFoundError(f"Video file not found at: {video_path}")

    # Intext Comment: Initialize video capture object from the video file path.
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise IOError(f"Could not open video file: {video_path}")

    try:
        # Intext Comment: Get the total number of frames in the video.
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if total_frames == 0:
            raise ValueError("Video file contains no frames.")

        # Intext Comment: Set the capture position to the last frame (index is total_frames - 1).
        cap.set(cv2.CAP_PROP_POS_FRAMES, total_frames - 1)

        # Intext Comment: Read the frame at the current position.
        success, frame = cap.read()
        if not success or frame is None:
            raise IOError(f"Failed to read the last frame from {video_path}.")

        # Intext Comment: Encode the frame (NumPy array) into a PNG image in memory.
        # The 'imencode' function returns a tuple (success_flag, buffer).
        success, buffer = cv2.imencode('.png', frame)
        if not success:
            raise RuntimeError("Failed to encode frame into PNG format.")

        # Intext Comment: Convert the memory buffer to bytes.
        image_bytes = buffer.tobytes()

        # Intext Comment: Encode the bytes into a base64 string and decode to UTF-8 for API compatibility.
        return base64.b64encode(image_bytes).decode('utf-8')

    finally:
        # Intext Comment: Always release the video capture object to free up resources.
        cap.release()


def _generate_video_from_veo(
    project_id: str,
    location: str,
    input_image_base64: str,
    prompt: str
) -> bytes:
    """
    Calls the Vertex AI VEO model to generate a video.

    Args:
        project_id: The Google Cloud project ID.
        location: The Google Cloud location (e.g., "us-central1").
        input_image_base64: The base64 encoded input image.
        prompt: The text prompt for video generation.

    Returns:
        The raw bytes of the generated video.

    Raises:
        google_exceptions.GoogleAPICallError: If the API call fails.
    """
    # Intext Comment: Initialize the Vertex AI SDK.
    aiplatform.init(project=project_id, location=location)

    # Intext Comment: Define the model endpoint for VEO.
    # Note: "veo-2" is a placeholder for the actual model name.
    # You must replace this with the official model identifier when available.
    model = aiplatform.Model("projects/vertex-ai-vision-public-preview/locations/us-central1/models/veo-2-0605")

    # Intext Comment: Construct the instance payload for the model prediction.
    # The exact format may vary based on the final VEO API specification.
    instance = {
        "prompt": prompt,
        "image_bytes": input_image_base64,
        # Intext Comment: Additional parameters can be added here, e.g., video length.
        "video_length_sec": 4, # Example parameter
    }

    # Intext Comment: Make the prediction call to the Vertex AI endpoint.
    # The response is expected to contain the video content, likely base64 encoded.
    response = model.predict(instances=[instance])

    # Intext Comment: Extract the base64 encoded video content from the first prediction.
    # The key 'video_bytes' is an assumption based on typical Vertex AI responses.
    video_base64 = response.predictions[0]['video_bytes']

    # Intext Comment: Decode the base64 string to get the raw video bytes.
    return base64.b64decode(video_base64)


# --- Main Function ---

def create_generative_video_story(
    reference_image_path: str,
    prompts: List[str],
    gcp_project_id: str,
    gcp_location: str,
    output_folder_name: str
) -> Path:
    """
    Creates a continuous video story from a reference image and a series of prompts using Google's VEO-2 model.

    This function performs the following steps:
    1. Creates a dedicated output directory.
    2. Loads an initial reference image.
    3. Iteratively generates video segments:
       - The first segment is based on the reference image and the first prompt.
       - Each subsequent segment is based on the last frame of the previous video and the next prompt.
    4. Saves each video segment sequentially.
    5. Stitches all segments together into a final, seamless video.

    Args:
        reference_image_path (str): The full path to the starting reference image file.
        prompts (List[str]): A list of strings, where each string is a prompt for a scene.
        gcp_project_id (str): Your Google Cloud Project ID.
        gcp_location (str): The Google Cloud location for the Vertex AI API call (e.g., "us-central1").
        output_folder_name (str): The name for the sub-directory where generated videos will be saved.

    Returns:
        Path: The path to the final stitched video file.

    Raises:
        ValueError: If the prompts list is empty or inputs are invalid.
        FileNotFoundError: If the reference image file does not exist.
        IOError: If there are issues reading files or writing video.
        google_exceptions.GoogleAPICallError: If a Vertex AI API call fails.
    """
    # --- 0. Input Validation ---
    # Intext Comment: Ensure the reference image path is a valid string.
    if not isinstance(reference_image_path, str) or not reference_image_path:
        raise ValueError("The 'reference_image_path' must be a non-empty string.")
    # Intext Comment: Ensure the prompts list is a non-empty list of strings.
    if not prompts or not isinstance(prompts, list) or not all(isinstance(p, str) for p in prompts):
        raise ValueError("The 'prompts' must be a non-empty list of strings.")
    # Intext Comment: Ensure GCP and folder name inputs are valid strings.
    if not isinstance(gcp_project_id, str) or not gcp_project_id:
        raise ValueError("The 'gcp_project_id' must be a non-empty string.")
    if not isinstance(gcp_location, str) or not gcp_location:
        raise ValueError("The 'gcp_location' must be a non-empty string.")
    if not isinstance(output_folder_name, str) or not output_folder_name:
        raise ValueError("The 'output_folder_name' must be a non-empty string.")

    # --- 1. Create Output Sub-directory ---
    try:
        # Intext Comment: Get the user's home directory and create the full path for the output folder.
        output_dir = Path.home() / output_folder_name
        # Intext Comment: Create the directory. `exist_ok=True` prevents an error if it already exists.
        output_dir.mkdir(parents=True, exist_ok=True)
        print(f"Successfully created or found output directory: {output_dir}")
    except PermissionError:
        # Intext Comment: Handle cases where the script doesn't have permission to create the directory.
        raise PermissionError(f"Permission denied to create directory at {output_dir}.")

    # --- 2. Read and Prepare the Initial Reference Image ---
    try:
        # Intext Comment: Open the image file using Pillow.
        with Image.open(reference_image_path) as img:
            # Intext Comment: Convert to RGB to ensure consistency and handle formats like PNG with alpha channels.
            img_rgb = img.convert('RGB')
            # Intext Comment: Create an in-memory binary stream to save the image.
            from io import BytesIO
            buffer = BytesIO()
            # Intext Comment: Save the image into the buffer in JPEG format.
            img_rgb.save(buffer, format="JPEG")
            # Intext Comment: Get the byte value of the buffer.
            image_bytes = buffer.getvalue()
            # Intext Comment: Encode the image bytes to a base64 string for the API.
            current_image_base64 = base64.b64encode(image_bytes).decode('utf-8')
            print(f"Successfully read and encoded reference image: {reference_image_path}")
    except FileNotFoundError:
        # Intext Comment: Handle the case where the image file does not exist.
        raise FileNotFoundError(f"Reference image not found at: {reference_image_path}")
    except UnidentifiedImageError:
        # Intext Comment: Handle cases where the file is not a valid image.
        raise IOError(f"The file at {reference_image_path} is not a valid or supported image format.")

    # --- 3. to 5. Iterate, Generate, Save, and Decompose ---
    generated_video_paths = []
    for i, prompt in enumerate(prompts, 1):
        print(f"\n--- Processing Prompt {i}/{len(prompts)}: '{prompt}' ---")
        try:
            # Intext Comment: Step 4 & 5c/f - Generate video using the current image and prompt.
            print("Generating video with VEO-2... (This may take some time)")
            video_bytes = _generate_video_from_veo(
                project_id=gcp_project_id,
                location=gcp_location,
                input_image_base64=current_image_base64,
                prompt=prompt
            )

            # Intext Comment: Step 5a/d - Save the generated video to the output folder.
            video_path = output_dir / f"{i}.mp4"
            with open(video_path, "wb") as f:
                f.write(video_bytes)
            generated_video_paths.append(video_path)
            print(f"Successfully saved video segment to: {video_path}")

            # Intext Comment: If this is not the last prompt, extract the last frame for the next iteration.
            if i < len(prompts):
                # Intext Comment: Step 5b/e - Decompose video and get the last frame as base64.
                print("Decomposing video to extract the last frame...")
                current_image_base64 = _get_last_frame_as_base64(video_path)
                print("Successfully extracted last frame for the next iteration.")

        except google_exceptions.GoogleAPICallError as e:
            print(f"An error occurred with the Vertex AI API on prompt {i}: {e}")
            raise
        except Exception as e:
            print(f"An unexpected error occurred during video generation for prompt {i}: {e}")
            raise

    # --- 6. Stitch All Videos Together ---
    if not generated_video_paths:
        print("No videos were generated, skipping final stitching.")
        return None

    print("\n--- Stitching all video segments into a final video ---")
    clips = []
    try:
        # Intext Comment: Create a list of VideoFileClip objects from the generated video paths.
        # The list is already in the correct order (1, 2, 3, ...).
        for video_path in generated_video_paths:
            clips.append(VideoFileClip(str(video_path)))

        # Intext Comment: Concatenate all the video clips into a single timeline.
        final_clip = concatenate_videoclips(clips, method="compose")

        # Intext Comment: Define the path for the final output video.
        final_video_path = output_dir / "final.mp4"

        # Intext Comment: Write the final concatenated video to a file.
        # `codec="libx264"` is a high-quality, widely compatible codec.
        # `audio_codec="aac"` is a standard audio codec.
        final_clip.write_videofile(
            str(final_video_path),
            codec="libx264",
            audio_codec="aac",
            temp_audiofile='temp-audio.m4a',
            remove_temp=True
        )
        print(f"Successfully stitched all segments. Final video saved to: {final_video_path}")
        return final_video_path

    except Exception as e:
        # Intext Comment: Handle potential errors during the moviepy processing.
        print(f"An error occurred during the final video stitching process: {e}")
        raise
    finally:
        # Intext Comment: Best practice: explicitly close all clip objects to release file handles.
        for clip in clips:
            clip.close()


In [ ]:
### **The Complete Professional-Grade Python Function**

import os
from pathlib import Path
from typing import List
from PIL import Image, UnidentifiedImageError
import cv2
import numpy as np
import vertexai
from vertexai.preview.vision_models import VideoGenerationModel, Video
from moviepy.editor import VideoFileClip, concatenate_videoclips

def generate_sequential_video_from_prompts(
    reference_image_path: str,
    scene_prompts: List[str],
    vertex_project_id: str,
    vertex_location: str,
    vertex_api_key: str,
    output_folder_name: str,
) -> None:
    """
    Generates a sequential video from a reference image and a list of prompts.

    This function performs the following steps:
    1. Creates a subdirectory in the home directory for the output.
    2. Reads the initial reference image.
    3. Iterates through the prompts, generating a video for each.
    4. The first video is generated from the reference image.
    5. Subsequent videos are generated from the last frame of the previous video.
    6. Each generated video is saved sequentially.
    7. Finally, all generated videos are stitched into a single "final.mp4" video.

    Args:
        reference_image_path: Path to the initial reference image.
        scene_prompts: A list of strings, where each string is a prompt for a scene.
        vertex_project_id: Your Google Cloud project ID.
        vertex_location: The location of the Vertex AI resources (e.g., "us-central1").
        vertex_api_key: Your Google Vertex AI API key.
        output_folder_name: The name of the folder to save the generated videos.

    Raises:
        ValueError: If the list of scene prompts is empty.
        FileNotFoundError: If the reference image is not found.
        UnidentifiedImageError: If the reference file is not a valid image.
        Exception: For errors related to API calls, video processing, or file I/O.
    """
    # --- Input Validation ---
    if not scene_prompts:
        raise ValueError("The 'scene_prompts' list cannot be empty.")
    if not all(isinstance(p, str) for p in scene_prompts):
        raise TypeError("All elements in 'scene_prompts' must be strings.")
    if not isinstance(reference_image_path, str) or not reference_image_path:
        raise ValueError("'reference_image_path' must be a non-empty string.")
    # Add more validation for other parameters as needed.

    try:
        # --- Step 1: Create Output Directory ---
        print("Step 1: Creating output directory...")
        output_directory = create_output_directory(output_folder_name)
        print(f"Successfully created directory at: {output_directory}")

        # --- Step 2: Read Reference Image ---
        print("Step 2: Reading reference image...")
        current_frame = load_reference_image(reference_image_path)
        print(f"Successfully loaded reference image from: {reference_image_path}")

        # --- Initialize Vertex AI ---
        print("Initializing Vertex AI...")
        vertexai.init(project=vertex_project_id, location=vertex_location, credentials=vertex_api_key)
        model = VideoGenerationModel.from_pretrained("video-generation-001")
        print("Vertex AI initialized and model loaded.")

        # --- Steps 3, 4, & 5: Iterate, Generate, Save, Decompose ---
        print("Starting video generation loop...")
        for i, prompt in enumerate(scene_prompts):
            iteration_num = i + 1
            print(f"\n--- Iteration {iteration_num}/{len(scene_prompts)} ---")
            print(f"Prompt: '{prompt}'")

            # --- Generate Video ---
            print("Generating video with Veo-2...")
            video_generation_response = model.generate(
                prompt=prompt,
                image=current_frame,
            )
            generated_video = video_generation_response.videos[0]

            # --- Save Generated Video ---
            video_filename = f"{iteration_num}.mp4"
            video_path = output_directory / video_filename
            print(f"Saving video to: {video_path}")
            generated_video.save(str(video_path))

            # --- Decompose and Get Last Frame for Next Iteration ---
            if iteration_num < len(scene_prompts):
                print("Decomposing video to get the last frame for the next iteration...")
                current_frame = decompose_video_and_get_last_frame(video_path)
                print("Successfully extracted last frame.")

        print("\nAll video segments have been generated.")

        # --- Step 6: Stitch Videos ---
        print("\nStep 6: Stitching all video segments together...")
        stitch_videos(output_directory, "final")
        print(f"Successfully stitched videos. Final video saved as 'final.mp4' in '{output_directory}'")

    except (ValueError, TypeError, FileNotFoundError, UnidentifiedImageError) as e:
        print(f"An error occurred: {e}")
        # Re-raise the exception after logging it.
        raise
    except Exception as e:
        # Catch any other exceptions (API errors, etc.)
        print(f"An unexpected error occurred: {e}")
        raise

#### **Step 1: Create a Sub-directory**

def create_output_directory(folder_name: str) -> Path:
    """
    Creates a subdirectory in the user's home directory.

    Args:
        folder_name: The name of the directory to create.

    Returns:
        The path to the created directory.

    Raises:
        OSError: If the directory cannot be created.
    """
    try:
        # Construct the full path to the desired directory within the home directory.
        home_directory = Path.home()
        output_directory = home_directory / folder_name
        # Create the directory. The `exist_ok=True` argument prevents an error
        # if the directory already exists.
        output_directory.mkdir(parents=True, exist_ok=True)
        return output_directory
    except OSError as e:
        # Handle potential OS-level errors during directory creation.
        print(f"Error creating directory: {e}")
        raise


#### **Step 2: Read the Reference Image**


def load_reference_image(image_path: str) -> Image.Image:
    """
    Loads an image from the specified path into a PIL Image object.

    Args:
        image_path: The full path to the reference image file.

    Returns:
        A PIL Image object.

    Raises:
        FileNotFoundError: If the image file does not exist.
        UnidentifiedImageError: If the file is not a valid image.
    """
    try:
        # Open the image file using a 'with' statement for automatic resource management.
        with Image.open(image_path) as img:
            # Load the image data into memory.
            img.load()
            return img
    except FileNotFoundError:
        # Handle the case where the file does not exist.
        print(f"Error: The image file was not found at '{image_path}'")
        raise
    except UnidentifiedImageError:
        # Handle the case where the file is not a valid image.
        print(f"Error: The file at '{image_path}' is not a valid image.")
        raise

#### **Step 3 & 4: Iterate and Generate the First Video**

def generate_initial_video(
    project_id: str,
    location: str,
    reference_image: Image.Image,
    prompt: str,
    api_key: str
) -> Video:
    """
    Generates the initial video from a reference image and a prompt.

    Args:
        project_id: The Google Cloud project ID.
        location: The Google Cloud location.
        reference_image: The reference PIL Image.
        prompt: The text prompt for the video generation.
        api_key: The Vertex AI API key.

    Returns:
        The generated Video object.
    """
    # Initialize Vertex AI
    vertexai.init(project=project_id, location=location, credentials=api_key)

    # Load the model
    model = VideoGenerationModel.from_pretrained("video-generation-001")

    # Generate the video
    video_generation_response = model.generate(
        prompt=prompt,
        image=reference_image,
    )
    return video_generation_response.videos[0]


#### **Step 5: Save, Decompose, and Iterate**

def save_video(video: Video, output_path: Path):
    """
    Saves a Vertex AI Video object to a file.

    Args:
        video: The Video object to save.
        output_path: The path to save the video file.
    """
    try:
        # Save the video to the specified path.
        video.save(str(output_path))
    except Exception as e:
        # Handle potential errors during saving.
        print(f"Error saving video to {output_path}: {e}")
        raise

def decompose_video_and_get_last_frame(video_path: Path) -> Image.Image:
    """
    Decomposes a video and returns the last frame as a PIL Image.

    Args:
        video_path: The path to the video file.

    Returns:
        The last frame of the video as a PIL Image.

    Raises:
        ValueError: If the video cannot be opened or has no frames.
    """
    try:
        # Open the video file.
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            raise ValueError(f"Could not open video file: {video_path}")

        last_frame = None
        while True:
            # Read the next frame.
            ret, frame = cap.read()
            if not ret:
                # Break the loop if there are no more frames.
                break
            last_frame = frame

        # Release the video capture object.
        cap.release()

        if last_frame is None:
            raise ValueError(f"Video has no frames: {video_path}")

        # Convert the last frame from BGR (OpenCV format) to RGB (PIL format).
        last_frame_rgb = cv2.cvtColor(last_frame, cv2.COLOR_BGR2RGB)
        # Create a PIL Image from the NumPy array.
        return Image.fromarray(last_frame_rgb)
    except cv2.error as e:
        # Handle OpenCV-specific errors.
        print(f"OpenCV error processing video {video_path}: {e}")
        raise

def generate_subsequent_video(
    project_id: str,
    location: str,
    last_frame: Image.Image,
    prompt: str,
    api_key: str
) -> Video:
    """
    Generates a subsequent video from a reference frame and a prompt.

    Args:
        project_id: The Google Cloud project ID.
        location: The Google Cloud location.
        last_frame: The reference PIL Image (last frame of the previous video).
        prompt: The text prompt for the video generation.
        api_key: The Vertex AI API key.

    Returns:
        The generated Video object.
    """
    # This function is conceptually the same as generate_initial_video,
    # but is separated for clarity in the main loop.
    return generate_initial_video(project_id, location, last_frame, prompt, api_key)

#### **Step 6: Stitch Videos**

def stitch_videos(video_directory: Path, final_video_name: str):
    """
    Stitches all videos in a directory into a single video.

    Args:
        video_directory: The directory containing the video files.
        final_video_name: The name for the final stitched video (without extension).
    """
    try:
        # Get all .mp4 files in the directory.
        video_files = sorted(
            [f for f in video_directory.glob("*.mp4") if f.stem.isdigit()],
            key=lambda f: int(f.stem)
        )

        if not video_files:
            print("No video files found to stitch.")
            return

        # Create a list of VideoFileClip objects.
        clips = [VideoFileClip(str(f)) for f in video_files]

        # Concatenate the video clips.
        final_clip = concatenate_videoclips(clips)

        # Write the final video file.
        final_output_path = video_directory / f"{final_video_name}.mp4"
        final_clip.write_videofile(str(final_output_path), codec="libx264")

        # Close the clips to release resources.
        for clip in clips:
            clip.close()
        final_clip.close()

    except Exception as e:
        # Handle potential errors during video stitching.
        print(f"Error stitching videos: {e}")
        raise


# Gemini 2.5 Pro May Draft

In [ ]:
import os
import base64
from pathlib import Path
from typing import List, Dict, Any, Tuple

from PIL import Image as PILImage # Used for initial image validation, if desired
import cv2 # OpenCV for video frame extraction
from moviepy.editor import VideoFileClip, concatenate_videoclips # For stitching videos

# Google Cloud specific imports
from google.cloud import aiplatform_v1beta1 as aiplatform
from google.protobuf import json_format, struct_pb2
from google.api_core.client_options import ClientOptions
# Note: For google.auth, typical ADC usage doesn't require explicit imports here,
# but if using service account keys directly:
# from google.oauth2.service_account import Credentials

def generate_animated_video_sequence(
    reference_image_file_path: str,
    prompts: List[str],
    gcp_project_id: str,
    gcp_location: str,
    google_vertex_api_key: str,
    output_folder_name: str,
    veo_model_id: str = "video-generation-001" # Example model ID, replace with actual VEO-2 ID
) -> str:
    """
    Generates a sequence of animated videos based on a reference image and prompts,
    using a Google Vertex AI video generation model (e.g., VEO-2).

    The process involves:
    1. Creating an output directory.
    2. Reading an initial reference image.
    3. Iteratively generating video segments:
        - The first segment uses the initial image and the first prompt.
        - Subsequent segments use the last frame of the previously generated
          video and the next prompt.
        - Each segment is saved numerically (1.mp4, 2.mp4, ...).
    4. Stitching all generated segments into a final video named "final.mp4".

    Args:
        reference_image_file_path: Path to the initial reference image file.
        prompts: A list of text prompts, one for each video segment.
        gcp_project_id: Google Cloud Project ID.
        gcp_location: Google Cloud region for Vertex AI services (e.g., "us-central1").
        google_vertex_api_key: Google Vertex API Key. This will be passed to
                               ClientOptions. Standard Vertex AI authentication
                               (ADC/Service Account) is typically preferred.
        output_folder_name: Name of the sub-directory in the user's home directory
                            for storing generated videos.
        veo_model_id: The Vertex AI model ID for VEO-2 or a compatible video
                      generation model (e.g., "video-generation-001" for Imagen).

    Returns:
        The file path to the final stitched video.

    Raises:
        FileNotFoundError: If the reference image file does not exist.
        ValueError: If inputs are invalid (e.g., empty prompts list).
        RuntimeError: If any part of the video generation or processing fails.
        PermissionError: If directory creation fails due to permissions.
    """

    # --- Input Validation ---
    # Validate reference_image_file_path
    if not Path(reference_image_file_path).is_file():
        raise FileNotFoundError(f"Reference image file not found: {reference_image_file_path}")

    # Validate prompts
    if not prompts:
        raise ValueError("Prompts list cannot be empty.")
    if not all(isinstance(p, str) and p.strip() for p in prompts):
        raise ValueError("All prompts must be non-empty strings.")

    # Validate other essential string inputs
    for arg_name, arg_val in [
        ("gcp_project_id", gcp_project_id),
        ("gcp_location", gcp_location),
        ("output_folder_name", output_folder_name),
        ("veo_model_id", veo_model_id)
    ]: # google_vertex_api_key can be empty if auth handled differently
        if not isinstance(arg_val, str) or not arg_val.strip():
            raise ValueError(f"{arg_name} must be a non-empty string.")
    if not isinstance(google_vertex_api_key, str): # API key must be a string, even if empty
        raise ValueError("google_vertex_api_key must be a string.")


    # --- Step 1: Create output sub-directory ---
    # Construct the full path for the output directory in the home directory.
    home_dir = Path.home()
    # Create the full path to the output directory.
    output_dir = home_dir / output_folder_name
    try:
        # Create the output directory. `exist_ok=True` prevents an error if it already exists.
        os.makedirs(output_dir, exist_ok=True)
    except PermissionError:
        # Handle PermissionError if the script cannot create the directory.
        raise PermissionError(f"Permission denied to create directory: {output_dir}")
    except OSError as e:
        # Handle other OS-related errors during directory creation.
        raise RuntimeError(f"Failed to create output directory {output_dir}: {e}")

    # --- Initialize Vertex AI Prediction Client ---
    # Construct the API endpoint for the Vertex AI Prediction service.
    api_endpoint = f"{gcp_location}-aiplatform.googleapis.com"
    # Client options including the API key.
    # Note: Efficacy of API key depends on service and its auth configuration.
    client_options_args = {"api_endpoint": api_endpoint}
    if google_vertex_api_key: # Only add api_key if it's provided
        client_options_args["api_key"] = google_vertex_api_key
    client_options = ClientOptions(**client_options_args)

    try:
        # Initialize the Prediction Service client.
        prediction_client = aiplatform.PredictionServiceClient(client_options=client_options)
    except Exception as e:
        # Handle errors during Vertex AI client initialization.
        raise RuntimeError(f"Failed to initialize Vertex AI PredictionServiceClient: {e}. "
                           "Ensure authentication (ADC or API key) is correctly configured for the project/service.")

    # The full model resource name for Vertex AI prediction.
    # This assumes `veo_model_id` is a published model like "video-generation-001".
    model_resource_name = (
        f"projects/{gcp_project_id}/locations/{gcp_location}/publishers/google/models/{veo_model_id}"
    )

    # --- Helper Function: Generate video using Vertex AI Model ---
    def _generate_video_from_vertex_ai(
        image_bytes: bytes,
        prompt_text: str,
        video_length_sec: int = 5, # Default, VEO-2 might have different/more params
        fps: int = 24              # Default, VEO-2 might have different/more params
    ) -> bytes:
        # Encode image bytes to base64 for JSON payload.
        encoded_image_string = base64.b64encode(image_bytes).decode("utf-8")

        # Construct the instance payload for the prediction request.
        # THIS PAYLOAD IS AN EXAMPLE AND MUST MATCH THE VEO-2 MODEL'S ACTUAL API SCHEMA.
        instance_dict = {
            "prompt": prompt_text,
            "image": {"bytesBase64Encoded": encoded_image_string},
            # VEO-2 might require other specific fields here.
        }
        instance_pb = json_format.ParseDict(instance_dict, struct_pb2.Value())
        instances = [instance_pb]

        # Define model parameters. THESE ARE EXAMPLES AND MUST MATCH VEO-2's API.
        parameters_dict = {
            "videoLengthSec": video_length_sec, # Common naming: videoLength (camelCase)
            "fps": fps,
            # "safetyFilter": True, # Example boolean parameter
            # Add other VEO-2 specific parameters here.
        }
        parameters_pb = json_format.ParseDict(parameters_dict, struct_pb2.Value())

        try:
            # Make the prediction request to the Vertex AI model.
            response = prediction_client.predict(
                endpoint=model_resource_name,
                instances=instances,
                parameters=parameters_pb,
            )
            # Process the response. THIS PROCESSING LOGIC IS AN EXAMPLE.
            if not response.predictions:
                raise RuntimeError("Vertex AI API call returned no predictions.")

            # Prediction result parsing depends heavily on the model's output schema.
            prediction_result = json_format.MessageToDict(response.predictions[0])

            if "bytesBase64Encoded" in prediction_result: # Common for direct byte output
                video_bytes_b64 = prediction_result["bytesBase64Encoded"]
                generated_video_bytes = base64.b64decode(video_bytes_b64)
                return generated_video_bytes
            # VEO-2 might return a GCS URI instead, requiring download logic.
            elif "gcsUri" in prediction_result:
                gcs_uri = prediction_result["gcsUri"]
                raise NotImplementedError(
                    f"Video generation resulted in a GCS URI ({gcs_uri}). "
                    "Direct GCS download is not implemented. Assumed direct byte output."
                )
            else:
                raise RuntimeError(f"Unexpected API response format. Prediction: {prediction_result}")
        except Exception as e:
            # Handle errors during the API call.
            raise RuntimeError(f"Vertex AI API call failed for prompt '{prompt_text}': {e}")

    # --- Helper Function: Decompose video and get last frame ---
    def _get_last_frame_from_video(video_file_path: str) -> bytes:
        try:
            # Open the video file using OpenCV.
            cap = cv2.VideoCapture(video_file_path)
            if not cap.isOpened():
                raise RuntimeError(f"OpenCV: Failed to open video file: {video_file_path}")

            # Get the total number of frames.
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            if total_frames == 0:
                cap.release()
                raise RuntimeError(f"OpenCV: Video file has no frames: {video_file_path}")

            # Set video capture position to the last frame (0-indexed).
            cap.set(cv2.CAP_PROP_POS_FRAMES, total_frames - 1)
            # Read the last frame.
            ret, frame = cap.read()

            if not ret or frame is None:
                cap.release()
                raise RuntimeError(f"OpenCV: Failed to read last frame from: {video_file_path}")

            # Release video capture object.
            cap.release()

            # Encode frame (numpy array) to JPEG image bytes.
            success, encoded_image = cv2.imencode(".jpg", frame, [cv2.IMWRITE_JPEG_QUALITY, 90])
            if not success:
                raise RuntimeError(f"OpenCV: Failed to encode last frame to JPEG from: {video_file_path}")

            return encoded_image.tobytes()
        except Exception as e:
            # Catch-all for OpenCV or processing errors.
            if 'cap' in locals() and cap.isOpened():
                cap.release()
            raise RuntimeError(f"OpenCV: Error processing video {video_file_path} for last frame: {e}")

    # --- Main Processing Loop ---
    generated_video_segment_paths: List[str] = []
    current_input_image_bytes: bytes

    # --- Step 2: Read the initial reference image into memory ---
    try:
        # Read the initial reference image file as bytes.
        current_input_image_bytes = Path(reference_image_file_path).read_bytes()
    except Exception as e:
        # Handle potential errors during file reading.
        raise RuntimeError(f"Failed to read reference image file {reference_image_file_path}: {e}")

    # --- Step 3 & onwards: Iterate through prompts, generate videos, extract frames ---
    for idx, prompt_text in enumerate(prompts):
        # Determine if this is the first iteration for logging/logic if needed.
        # is_first_iteration = (idx == 0) (not strictly used below but good for clarity)

        # --- Step 4 (First Iteration) / Step 5.c (Subsequent Iterations): Generate video ---
        try:
            # Call helper to generate video using current input image and prompt.
            generated_video_bytes = _generate_video_from_vertex_ai(
                image_bytes=current_input_image_bytes,
                prompt_text=prompt_text
                # Pass other VEO-2 specific parameters if needed
            )
        except Exception as e:
            # Propagate error from video generation.
            raise RuntimeError(f"Video generation failed for prompt #{idx + 1} ('{prompt_text}'): {e}")

        # --- Step 5.a / 5.d: Save the generated video ---
        # Filename uses 1-based indexing.
        video_segment_filename = f"{idx + 1}.mp4" # Assuming MP4 format from VEO-2.
        video_segment_filepath = output_dir / video_segment_filename
        try:
            # Write the generated video bytes to file.
            with open(video_segment_filepath, "wb") as f:
                f.write(generated_video_bytes)
            # Add path of saved segment for later stitching.
            generated_video_segment_paths.append(str(video_segment_filepath))
        except IOError as e:
            # Handle errors during file saving.
            raise RuntimeError(f"Failed to save video segment {video_segment_filepath}: {e}")

        # --- Step 5.b / 5.e: Decompose video, get last frame (if not the last prompt) ---
        # This is needed to feed into the *next* iteration.
        if idx < len(prompts) - 1: # If there's a next prompt
            try:
                # Get last frame from the just-saved video.
                last_frame_bytes = _get_last_frame_from_video(str(video_segment_filepath))
                # Update current_input_image_bytes for the next iteration.
                current_input_image_bytes = last_frame_bytes
            except Exception as e:
                # Propagate error from frame extraction.
                raise RuntimeError(f"Failed to extract last frame from {video_segment_filepath}: {e}")
        # Step 5.f and 5.g are implicitly handled by this loop structure.

    # --- Step 6: Stitch all generated video segments together ---
    if not generated_video_segment_paths:
        # This should ideally not be reached due to prior prompt validation.
        raise RuntimeError("No video segments were generated; cannot stitch.")

    final_video_filename = "final.mp4"
    final_video_filepath = output_dir / final_video_filename

    video_clips_to_stitch: List[VideoFileClip] = []
    try:
        # Load each generated video segment as a MoviePy VideoFileClip.
        for segment_path_str in generated_video_segment_paths:
            try:
                # Load the video clip.
                clip = VideoFileClip(segment_path_str)
                video_clips_to_stitch.append(clip)
            except Exception as e:
                # Handle error if a segment cannot be loaded.
                raise RuntimeError(f"MoviePy: Failed to load video segment {segment_path_str} for stitching: {e}")

        if not video_clips_to_stitch:
            raise RuntimeError("MoviePy: No video clips successfully loaded for stitching.")

        # Concatenate all video clips.
        final_clip = concatenate_videoclips(video_clips_to_stitch, method="compose")

        # Write the final stitched video to file.
        # Ensure VEO-2 generated videos have consistent FPS, resolution for best results.
        # Specify codec, threads, logger for `write_videofile` as needed.
        final_clip.write_videofile(
            str(final_video_filepath),
            codec="libx264", # Common, good quality MP4 codec
            # audio_codec="aac", # Add if VEO-2 includes audio and it should be preserved
            # threads=os.cpu_count(), # Utilize available CPU cores for encoding
            # logger="bar" # Shows a progress bar
        )
        # Return the path to the final video.
        return str(final_video_filepath)

    except Exception as e:
        # Handle errors during video stitching or saving.
        raise RuntimeError(f"MoviePy: Failed to stitch video segments or save final video: {e}")
    finally:
        # Clean up: Close all loaded MoviePy clips.
        for clip_obj in video_clips_to_stitch:
            try:
                clip_obj.close()
            except Exception:
                # Log or ignore errors during cleanup of individual clips.
                # print(f"Warning: MoviePy: Failed to close a video clip resource for {clip_obj.filename}")
                pass


## Sonnet 4 *Draft*

## Draft 1

### Assessment

# VEO-2 Video Generation Pipeline Code Analysis

## Overview
This is a comprehensive analysis of the VEO-2 video generation pipeline implementation. I'll examine each section for accuracy, best practices, and potential issues.

## Import Section Analysis

### ✅ Standard Library Imports - CORRECT
```python
import os
import sys
import time
import json
import base64
import logging
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple, Union
```
- All standard library imports are correct and properly used throughout the code
- Type hints imports are appropriate for the code's type annotations

### ⚠️ Third-Party Imports - MIXED ACCURACY
```python
import cv2
import numpy as np
from PIL import Image
import requests
from google.auth import default
from google.auth.transport.requests import Request
from google.cloud import storage
import moviepy.editor as mp
from moviepy.editor import VideoFileClip, concatenate_videoclips
import tempfile
import shutil
```

**Issues Found:**
1. `import tempfile` and `import shutil` are imported but never used in the code
2. `import moviepy.editor as mp` is redundant since specific functions are imported on the next line
3. Missing error handling for import failures of optional dependencies

## Logging Configuration - ✅ CORRECT
```python
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)
```
- Professional logging setup is appropriate and well-configured

## Class Definition and Initialization

### ✅ Exception Class - CORRECT
```python
class VEO2VideoGenerationError(Exception):
    """Custom exception for VEO-2 video generation errors."""
    pass
```
- Simple, appropriate custom exception class

### ✅ Main Class Initialization - MOSTLY CORRECT
```python
def __init__(self, project_id: str, location: str = "us-central1"):
```
**Analysis:**
- Parameter validation is good
- Instance variable initialization is appropriate
- API endpoint construction is correct for Vertex AI

**Minor Issue:** The model ID `"veo-2.0-generate-001"` should be verified against actual Vertex AI model naming conventions.

## Authentication Method

### ⚠️ Authentication Issues Found
```python
def _initialize_authentication(self) -> None:
    try:
        self.credentials, _ = default()
        if not self.credentials.valid:
            self.credentials.refresh(Request())
    except Exception as e:
        raise VEO2VideoGenerationError(f"Failed to initialize authentication: {str(e)}")
```

**Issues:**
1. The `default()` function returns `(credentials, project_id)`, but the project_id is ignored with `_`
2. Should handle the case where credentials are None
3. Missing import for specific Google Auth exceptions

## Input Validation

### ✅ Validation Logic - EXCELLENT
```python
def _validate_inputs(self, reference_image_path: str, scene_prompts: List[str],
                    api_key: str, output_folder_name: str) -> None:
```
- Comprehensive validation of all input parameters
- Proper type checking and existence verification
- Good error messages

## Directory Creation

### ✅ Directory Creation - CORRECT
```python
def _create_output_directory(self, folder_name: str) -> Path:
    home_dir = Path.home()
    output_dir = home_dir / folder_name
    output_dir.mkdir(mode=0o755, parents=True, exist_ok=True)
```
- Uses pathlib appropriately
- Proper permissions and error handling

## Image Processing Methods

### ✅ Image Loading and Encoding - WELL IMPLEMENTED
```python
def _load_and_encode_image(self, image_path: str) -> Tuple[str, str]:
```
**Strengths:**
- Proper PIL usage for image handling
- Format conversion and base64 encoding is correct
- Good error handling

### ✅ Image Resizing - CORRECT
```python
def _resize_image_for_veo2(self, img: Image.Image) -> Image.Image:
```
- Aspect ratio calculation is correct
- Uses high-quality resampling (LANCZOS)
- Appropriate target dimensions

## Critical Issues in API Integration

### ❌ MAJOR ISSUES in VEO-2 API Call
```python
def _generate_video_with_veo2(self, prompt: str, image_data: Optional[str] = None,
                             mime_type: Optional[str] = None, storage_uri: Optional[str] = None) -> str:
```

**Critical Problems:**

1. **Incorrect API Endpoint Structure:**
   ```python
   url = (f"{self.api_endpoint}/v1/projects/{self.project_id}/"
          f"locations/{self.location}/publishers/google/"
          f"models/{self.model_id}:predictLongRunning")
   ```
   This endpoint structure is incorrect for VEO-2. The actual Vertex AI endpoint should be different.

2. **Wrong Request Body Format:**
   ```python
   request_body = {
       "instances": [{
           "prompt": prompt
       }],
       "parameters": {
           "durationSeconds": 8,
           "sampleCount": 1,
           "aspectRatio": "16:9"
       }
   }
   ```
   This format doesn't match VEO-2's actual API specification.

3. **Authentication Header Issue:**
   ```python
   headers = {
       'Authorization': f'Bearer {self.credentials.token}',
       'Content-Type': 'application/json; charset=utf-8'
   }
   ```
   Should ensure credentials are refreshed and handle token expiration.

## Polling Operation

### ⚠️ Polling Logic - MOSTLY CORRECT BUT ISSUES
```python
def _poll_operation_status(self, operation_name: str) -> str:
```

**Issues:**
1. **Incorrect Polling Endpoint:**
   ```python
   url = (f"{self.api_endpoint}/v1/projects/{self.project_id}/"
          f"locations/{self.location}/publishers/google/"
          f"models/{self.model_id}:fetchPredictOperation")
   ```
   This endpoint structure is not correct for standard Google Cloud operations.

2. **Should use standard Google Cloud Operations API:**
   ```python
   # Should be something like:
   url = f"{self.api_endpoint}/v1/{operation_name}"
   ```

## File Download Method

### ✅ GCS Download - CORRECT
```python
def _download_video_from_gcs(self, gcs_uri: str, local_path: Path) -> None:
```
- URI parsing is correct
- Google Cloud Storage client usage is appropriate
- Error handling is good

## Video Processing Methods

### ✅ Frame Extraction - CORRECT
```python
def _extract_last_frame(self, video_path: Path) -> np.ndarray:
```
- OpenCV usage is correct
- Proper resource cleanup with `cap.release()`
- Good error handling

### ✅ Frame Conversion - CORRECT
```python
def _frame_to_base64(self, frame: np.ndarray) -> Tuple[str, str]:
```
- BGR to RGB conversion is necessary and correct
- PIL integration is appropriate
- Base64 encoding is implemented correctly

### ✅ Video Stitching - CORRECT
```python
def _stitch_videos(self, video_paths: List[Path], output_path: Path) -> None:
```
- MoviePy usage is correct
- Proper resource cleanup with clip.close()
- Good codec settings for output

## Main Pipeline Method

### ⚠️ Pipeline Logic - GOOD STRUCTURE, API ISSUES
```python
def generate_sequential_videos(self, ...):
```

**Strengths:**
- Well-structured pipeline flow
- Good error handling and logging
- Proper iteration through prompts with frame chaining

**Issues:**
- Inherits all the API-related problems from the helper methods
- The api_key parameter is accepted but not used (relies on default authentication)

## Helper Function

### ⚠️ Auto-detection Logic Issues
```python
def create_veo2_video_pipeline(...):
    if project_id is None:
        try:
            _, project_id = default()  # This is incorrect
```

**Critical Issue:** The `default()` function returns `(credentials, project_id)`, but in most cases, the project_id returned is None. The code should use:
```python
from google.auth import default
import google.auth.exceptions

credentials, project_id = default()
if not project_id:
    project_id = os.environ.get('GOOGLE_CLOUD_PROJECT')
```

## Summary of Critical Issues

### 🔴 Major Problems:
1. **VEO-2 API Integration**: The API endpoints and request formats are incorrect
2. **Authentication Flow**: Improper handling of project ID detection
3. **Unused Parameters**: api_key parameter is not utilized
4. **Import Redundancy**: Unused imports and redundant moviepy imports

### 🟡 Minor Issues:
1. Missing specific exception handling for Google Cloud errors
2. Model ID naming should be verified
3. Some parameters could have better validation

### ✅ Well-Implemented Areas:
1. Image processing and encoding
2. Video file operations (extraction, stitching)
3. File system operations
4. Error handling structure
5. Logging implementation
6. Type annotations and documentation

## Recommendations for Fixes:

1. **Fix API Integration**: Research and implement correct VEO-2 API endpoints and request formats
2. **Fix Authentication**: Properly handle project ID detection and credential management
3. **Remove Unused Code**: Clean up unused imports and parameters
4. **Add Specific Exception Handling**: Handle Google Cloud specific exceptions
5. **Verify Model Naming**: Confirm correct VEO-2 model identifier
6. **Integration Testing**: The code needs testing with actual VEO-2 API to verify correctness

The code shows excellent software engineering practices in terms of structure, documentation, and error handling, but has critical issues in the core API integration that would prevent it from working with the actual VEO-2 service.

In [ ]:
"""
Professional Implementation-Grade VEO-2 Video Generation Pipeline
MIT PhD Computer Science Implementation with Google Vertex AI Integration

This module provides a comprehensive solution for generating sequential videos
using Google's VEO-2 model, with frame extraction and seamless stitching capabilities.

Author: MIT PhD Computer Science Professor
Specialization: Large Language Models, Prompting, Google Vertex AI
"""

import os
import sys
import time
import json
import base64
import logging
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple, Union
import cv2
import numpy as np
from PIL import Image
import requests
from google.auth import default
from google.auth.transport.requests import Request
from google.cloud import storage
import moviepy.editor as mp
from moviepy.editor import VideoFileClip, concatenate_videoclips
import tempfile
import shutil

# Configure logging for professional debugging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


class VEO2VideoGenerationError(Exception):
    """Custom exception for VEO-2 video generation errors."""
    pass


class VEO2VideoPipeline:
    """
    Professional implementation-grade class for VEO-2 video generation pipeline.

    This class handles the complete workflow of generating sequential videos using
    Google's VEO-2 model, including frame extraction and video stitching.
    """

    def __init__(self, project_id: str, location: str = "us-central1"):
        """
        Initialize the VEO-2 video generation pipeline.

        Args:
            project_id (str): Google Cloud project ID
            location (str): Vertex AI location, defaults to us-central1

        Raises:
            VEO2VideoGenerationError: If initialization fails
        """
        # Validate input parameters
        if not project_id or not isinstance(project_id, str):
            raise ValueError("project_id must be a non-empty string")

        # Initialize instance variables
        self.project_id: str = project_id
        self.location: str = location
        self.model_id: str = "veo-2.0-generate-001"
        self.api_endpoint: str = f"https://{location}-aiplatform.googleapis.com"

        # Initialize authentication
        self._initialize_authentication()

        logger.info(f"VEO2VideoPipeline initialized for project: {project_id}")

    def _initialize_authentication(self) -> None:
        """
        Initialize Google Cloud authentication credentials.

        Raises:
            VEO2VideoGenerationError: If authentication fails
        """
        try:
            # Obtain default credentials for Google Cloud authentication
            self.credentials, _ = default()

            # Refresh credentials if needed
            if not self.credentials.valid:
                self.credentials.refresh(Request())

        except Exception as e:
            raise VEO2VideoGenerationError(f"Failed to initialize authentication: {str(e)}")

    def _validate_inputs(
        self,
        reference_image_path: str,
        scene_prompts: List[str],
        api_key: str,
        output_folder_name: str
    ) -> None:
        """
        Validate all input parameters for the video generation pipeline.

        Args:
            reference_image_path (str): Path to reference image file
            scene_prompts (List[str]): List of scene description prompts
            api_key (str): Google Vertex AI API key
            output_folder_name (str): Name for output folder

        Raises:
            ValueError: If any input parameter is invalid
        """
        # Validate reference image path
        if not reference_image_path or not isinstance(reference_image_path, str):
            raise ValueError("reference_image_path must be a non-empty string")

        if not os.path.exists(reference_image_path):
            raise ValueError(f"Reference image file does not exist: {reference_image_path}")

        # Validate scene prompts
        if not scene_prompts or not isinstance(scene_prompts, list):
            raise ValueError("scene_prompts must be a non-empty list")

        if not all(isinstance(prompt, str) and prompt.strip() for prompt in scene_prompts):
            raise ValueError("All scene prompts must be non-empty strings")

        # Validate API key
        if not api_key or not isinstance(api_key, str):
            raise ValueError("api_key must be a non-empty string")

        # Validate output folder name
        if not output_folder_name or not isinstance(output_folder_name, str):
            raise ValueError("output_folder_name must be a non-empty string")

    def _create_output_directory(self, folder_name: str) -> Path:
        """
        Create a subdirectory in the home directory with the specified name.

        Args:
            folder_name (str): Name of the folder to create

        Returns:
            Path: Path object pointing to the created directory

        Raises:
            VEO2VideoGenerationError: If directory creation fails
        """
        try:
            # Get home directory path
            home_dir = Path.home()

            # Create subdirectory path
            output_dir = home_dir / folder_name

            # Create directory if it doesn't exist, with proper permissions
            output_dir.mkdir(mode=0o755, parents=True, exist_ok=True)

            logger.info(f"Created output directory: {output_dir}")
            return output_dir

        except Exception as e:
            raise VEO2VideoGenerationError(f"Failed to create output directory: {str(e)}")

    def _load_and_encode_image(self, image_path: str) -> Tuple[str, str]:
        """
        Load image from file path and encode it to base64 format for API usage.

        Args:
            image_path (str): Path to the image file

        Returns:
            Tuple[str, str]: Base64 encoded image data and MIME type

        Raises:
            VEO2VideoGenerationError: If image loading or encoding fails
        """
        try:
            # Open and validate image using PIL
            with Image.open(image_path) as img:
                # Convert to RGB if necessary (removes alpha channel)
                if img.mode != 'RGB':
                    img = img.convert('RGB')

                # Resize image to recommended dimensions (1280x720 or 720x1280)
                # while maintaining aspect ratio
                img = self._resize_image_for_veo2(img)

                # Convert PIL Image to bytes
                import io
                buffer = io.BytesIO()

                # Determine output format and MIME type
                original_format = img.format if img.format else 'JPEG'
                if original_format.upper() in ['JPEG', 'JPG']:
                    img.save(buffer, format='JPEG', quality=95)
                    mime_type = 'image/jpeg'
                else:
                    img.save(buffer, format='PNG')
                    mime_type = 'image/png'

                # Encode to base64
                image_bytes = buffer.getvalue()
                base64_encoded = base64.b64encode(image_bytes).decode('utf-8')

                logger.info(f"Successfully encoded image: {image_path}")
                return base64_encoded, mime_type

        except Exception as e:
            raise VEO2VideoGenerationError(f"Failed to load and encode image: {str(e)}")

    def _resize_image_for_veo2(self, img: Image.Image) -> Image.Image:
        """
        Resize image to VEO-2 recommended dimensions while preserving aspect ratio.

        Args:
            img (Image.Image): PIL Image object

        Returns:
            Image.Image: Resized PIL Image object
        """
        # Get current dimensions
        width, height = img.size
        aspect_ratio = width / height

        # Determine target dimensions based on aspect ratio
        if aspect_ratio > 1:  # Landscape
            target_width, target_height = 1280, 720
        else:  # Portrait
            target_width, target_height = 720, 1280

        # Resize image using high-quality resampling
        resized_img = img.resize(
            (target_width, target_height),
            Image.Resampling.LANCZOS
        )

        return resized_img

    def _generate_video_with_veo2(
        self,
        prompt: str,
        image_data: Optional[str] = None,
        mime_type: Optional[str] = None,
        storage_uri: Optional[str] = None
    ) -> str:
        """
        Generate video using VEO-2 model via Vertex AI API.

        Args:
            prompt (str): Text prompt for video generation
            image_data (Optional[str]): Base64 encoded image data
            mime_type (Optional[str]): MIME type of the image
            storage_uri (Optional[str]): Cloud Storage URI for output

        Returns:
            str: Cloud Storage URI of generated video

        Raises:
            VEO2VideoGenerationError: If video generation fails
        """
        try:
            # Construct API endpoint URL
            url = (f"{self.api_endpoint}/v1/projects/{self.project_id}/"
                   f"locations/{self.location}/publishers/google/"
                   f"models/{self.model_id}:predictLongRunning")

            # Prepare request headers
            headers = {
                'Authorization': f'Bearer {self.credentials.token}',
                'Content-Type': 'application/json; charset=utf-8'
            }

            # Construct request body
            request_body = {
                "instances": [{
                    "prompt": prompt
                }],
                "parameters": {
                    "durationSeconds": 8,
                    "sampleCount": 1,
                    "aspectRatio": "16:9"
                }
            }

            # Add image data if provided
            if image_data and mime_type:
                request_body["instances"][0]["image"] = {
                    "bytesBase64Encoded": image_data,
                    "mimeType": mime_type
                }

            # Add storage URI if provided
            if storage_uri:
                request_body["parameters"]["storageUri"] = storage_uri

            # Send initial request
            response = requests.post(url, headers=headers, json=request_body)
            response.raise_for_status()

            # Extract operation name from response
            operation_data = response.json()
            operation_name = operation_data.get("name")

            if not operation_name:
                raise VEO2VideoGenerationError("No operation name returned from API")

            logger.info(f"Video generation started with operation: {operation_name}")

            # Poll operation status until completion
            video_uri = self._poll_operation_status(operation_name)

            return video_uri

        except requests.exceptions.RequestException as e:
            raise VEO2VideoGenerationError(f"API request failed: {str(e)}")
        except Exception as e:
            raise VEO2VideoGenerationError(f"Video generation failed: {str(e)}")

    def _poll_operation_status(self, operation_name: str) -> str:
        """
        Poll the status of a long-running video generation operation.

        Args:
            operation_name (str): Full operation name from initial request

        Returns:
            str: Cloud Storage URI of generated video

        Raises:
            VEO2VideoGenerationError: If polling fails or operation errors
        """
        try:
            # Construct polling endpoint URL
            url = (f"{self.api_endpoint}/v1/projects/{self.project_id}/"
                   f"locations/{self.location}/publishers/google/"
                   f"models/{self.model_id}:fetchPredictOperation")

            # Prepare request headers
            headers = {
                'Authorization': f'Bearer {self.credentials.token}',
                'Content-Type': 'application/json; charset=utf-8'
            }

            # Prepare request body
            request_body = {
                "operationName": operation_name
            }

            # Poll with exponential backoff
            max_attempts = 60  # Maximum 30 minutes (30 seconds * 60)
            attempt = 0
            wait_time = 30  # Start with 30 seconds

            while attempt < max_attempts:
                # Send polling request
                response = requests.post(url, headers=headers, json=request_body)
                response.raise_for_status()

                # Parse response
                result = response.json()

                # Check if operation is complete
                if result.get("done", False):
                    # Extract video URI from completed operation
                    response_data = result.get("response", {})
                    generated_samples = response_data.get("generatedSamples", [])

                    if not generated_samples:
                        raise VEO2VideoGenerationError("No generated samples in response")

                    video_info = generated_samples[0].get("video", {})
                    video_uri = video_info.get("uri")

                    if not video_uri:
                        raise VEO2VideoGenerationError("No video URI in response")

                    logger.info(f"Video generation completed: {video_uri}")
                    return video_uri

                # Wait before next poll
                time.sleep(wait_time)
                attempt += 1

                # Exponential backoff (cap at 120 seconds)
                wait_time = min(wait_time * 1.1, 120)

                logger.info(f"Polling attempt {attempt}/{max_attempts}, waiting {wait_time:.1f}s")

            raise VEO2VideoGenerationError("Operation timeout: video generation took too long")

        except requests.exceptions.RequestException as e:
            raise VEO2VideoGenerationError(f"Polling request failed: {str(e)}")
        except Exception as e:
            raise VEO2VideoGenerationError(f"Operation polling failed: {str(e)}")

    def _download_video_from_gcs(self, gcs_uri: str, local_path: Path) -> None:
        """
        Download video from Google Cloud Storage to local filesystem.

        Args:
            gcs_uri (str): Google Cloud Storage URI (gs://bucket/path)
            local_path (Path): Local file path for download

        Raises:
            VEO2VideoGenerationError: If download fails
        """
        try:
            # Parse GCS URI
            if not gcs_uri.startswith('gs://'):
                raise ValueError(f"Invalid GCS URI format: {gcs_uri}")

            # Extract bucket and blob name
            uri_parts = gcs_uri[5:].split('/', 1)  # Remove 'gs://' prefix
            bucket_name = uri_parts[0]
            blob_name = uri_parts[1] if len(uri_parts) > 1 else ''

            # Initialize GCS client
            storage_client = storage.Client(project=self.project_id, credentials=self.credentials)

            # Get bucket and blob
            bucket = storage_client.bucket(bucket_name)
            blob = bucket.blob(blob_name)

            # Download to local file
            blob.download_to_filename(str(local_path))

            logger.info(f"Downloaded video from {gcs_uri} to {local_path}")

        except Exception as e:
            raise VEO2VideoGenerationError(f"Failed to download video from GCS: {str(e)}")

    def _extract_last_frame(self, video_path: Path) -> np.ndarray:
        """
        Extract the last frame from a video file using OpenCV.

        Args:
            video_path (Path): Path to the video file

        Returns:
            np.ndarray: Last frame as numpy array

        Raises:
            VEO2VideoGenerationError: If frame extraction fails
        """
        try:
            # Open video capture
            cap = cv2.VideoCapture(str(video_path))

            if not cap.isOpened():
                raise VEO2VideoGenerationError(f"Cannot open video file: {video_path}")

            # Get total number of frames
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

            if total_frames == 0:
                raise VEO2VideoGenerationError(f"Video has no frames: {video_path}")

            # Seek to last frame
            cap.set(cv2.CAP_PROP_POS_FRAMES, total_frames - 1)

            # Read last frame
            ret, frame = cap.read()

            if not ret:
                raise VEO2VideoGenerationError(f"Cannot read last frame from: {video_path}")

            # Release video capture
            cap.release()

            logger.info(f"Extracted last frame from video: {video_path}")
            return frame

        except Exception as e:
            raise VEO2VideoGenerationError(f"Frame extraction failed: {str(e)}")

    def _frame_to_base64(self, frame: np.ndarray) -> Tuple[str, str]:
        """
        Convert OpenCV frame (numpy array) to base64 encoded string.

        Args:
            frame (np.ndarray): OpenCV frame as numpy array

        Returns:
            Tuple[str, str]: Base64 encoded image and MIME type

        Raises:
            VEO2VideoGenerationError: If conversion fails
        """
        try:
            # Convert BGR to RGB (OpenCV uses BGR, PIL uses RGB)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Convert to PIL Image
            pil_image = Image.fromarray(frame_rgb)

            # Resize for VEO-2 compatibility
            pil_image = self._resize_image_for_veo2(pil_image)

            # Convert to bytes
            import io
            buffer = io.BytesIO()
            pil_image.save(buffer, format='JPEG', quality=95)

            # Encode to base64
            image_bytes = buffer.getvalue()
            base64_encoded = base64.b64encode(image_bytes).decode('utf-8')

            return base64_encoded, 'image/jpeg'

        except Exception as e:
            raise VEO2VideoGenerationError(f"Frame to base64 conversion failed: {str(e)}")

    def _stitch_videos(self, video_paths: List[Path], output_path: Path) -> None:
        """
        Stitch multiple video files into a single seamless video.

        Args:
            video_paths (List[Path]): List of video file paths in sequence
            output_path (Path): Path for final stitched video

        Raises:
            VEO2VideoGenerationError: If video stitching fails
        """
        try:
            # Load all video clips
            video_clips = []

            for video_path in video_paths:
                if not video_path.exists():
                    raise VEO2VideoGenerationError(f"Video file not found: {video_path}")

                # Load video clip
                clip = VideoFileClip(str(video_path))
                video_clips.append(clip)

                logger.info(f"Loaded video clip: {video_path}")

            if not video_clips:
                raise VEO2VideoGenerationError("No video clips to stitch")

            # Concatenate all clips
            final_clip = concatenate_videoclips(video_clips, method="compose")

            # Write final video
            final_clip.write_videofile(
                str(output_path),
                codec='libx264',
                audio_codec='aac',
                temp_audiofile='temp-audio.m4a',
                remove_temp=True,
                verbose=False,
                logger=None
            )

            # Close all clips to free memory
            for clip in video_clips:
                clip.close()
            final_clip.close()

            logger.info(f"Successfully stitched {len(video_clips)} videos into: {output_path}")

        except Exception as e:
            # Ensure clips are closed on error
            for clip in video_clips:
                try:
                    clip.close()
                except:
                    pass

            raise VEO2VideoGenerationError(f"Video stitching failed: {str(e)}")

    def generate_sequential_videos(self,
        reference_image_path: str,
        scene_prompts: List[str],
        api_key: str,
        output_folder_name: str
    ) -> Path:
        """
        Main function to generate sequential videos using VEO-2 model.

        This function implements the complete pipeline as specified:
        1. Creates output subdirectory in home directory
        2. Loads and encodes reference image
        3. Iterates through scene prompts
        4. Generates videos using VEO-2 with frame chaining
        5. Stitches all videos into a final seamless video

        Args:
            reference_image_path (str): Directory location of reference image file
            scene_prompts (List[str]): List of scene description prompts
            api_key (str): Google Vertex AI API key (for compatibility, uses default auth)
            output_folder_name (str): Name for output folder in home directory

        Returns:
            Path: Path to the final stitched video file

        Raises:
            VEO2VideoGenerationError: If any step in the pipeline fails
            ValueError: If input validation fails
        """
        try:
            # Step 0: Validate all inputs
            self._validate_inputs(reference_image_path, scene_prompts, api_key, output_folder_name)

            # Step 1: Create subdirectory in home directory
            output_dir = self._create_output_directory(output_folder_name)

            # Step 2: Load and encode reference image
            initial_image_data, initial_mime_type = self._load_and_encode_image(reference_image_path)

            # Initialize variables for iteration
            current_image_data = initial_image_data
            current_mime_type = initial_mime_type
            video_paths = []

            # Step 3: Iterate through scene prompts
            for i, prompt in enumerate(scene_prompts):
                logger.info(f"Processing prompt {i+1}/{len(scene_prompts)}: {prompt[:50]}...")

                # Step 4: Generate video using current image and prompt
                video_uri = self._generate_video_with_veo2(
                    prompt=prompt,
                    image_data=current_image_data,
                    mime_type=current_mime_type
                )

                # Step 5a: Save generated video with enumerated name
                video_filename = f"{i+1}.mp4"
                video_path = output_dir / video_filename

                # Download video from GCS
                self._download_video_from_gcs(video_uri, video_path)
                video_paths.append(video_path)

                # Step 5b: Extract last frame for next iteration (if not last prompt)
                if i < len(scene_prompts) - 1:
                    last_frame = self._extract_last_frame(video_path)
                    current_image_data, current_mime_type = self._frame_to_base64(last_frame)

                    logger.info(f"Extracted last frame for next iteration")

            # Step 6: Stitch all videos into final seamless video
            final_video_path = output_dir / "final.mp4"
            self._stitch_videos(video_paths, final_video_path)

            logger.info(f"Sequential video generation completed successfully")
            logger.info(f"Final video saved to: {final_video_path}")

            return final_video_path

        except Exception as e:
            logger.error(f"Sequential video generation failed: {str(e)}")
            raise


def create_veo2_video_pipeline(
    reference_image_path: str,
    scene_prompts: List[str],
    api_key: str,
    output_folder_name: str,
    project_id: Optional[str] = None
) -> Path:
    """
    Professional implementation-grade function for VEO-2 sequential video generation.

    This function creates a complete video generation pipeline that:
    1. Creates a subdirectory in the home directory
    2. Processes a reference image for VEO-2 compatibility
    3. Generates sequential videos using frame chaining
    4. Stitches all videos into a seamless final output

    Args:
        reference_image_path (str): Directory location of reference image file
        scene_prompts (List[str]): List of scene description prompts
        api_key (str): Google Vertex AI API key
        output_folder_name (str): Name for output folder in home directory
        project_id (Optional[str]): Google Cloud project ID (auto-detected if None)

    Returns:
        Path: Path to the final stitched video file

    Raises:
        VEO2VideoGenerationError: If video generation pipeline fails
        ValueError: If input parameters are invalid

    Example:
        >>> final_video = create_veo2_video_pipeline(
        ...     reference_image_path="/path/to/image.jpg",
        ...     scene_prompts=[
        ...         "A bird takes flight from the tree",
        ...         "The bird soars over a mountain landscape",
        ...         "The bird lands near a peaceful lake"
        ...     ],
        ...     api_key="your-vertex-ai-key",
        ...     output_folder_name="my_video_project"
        ... )
        >>> print(f"Final video created: {final_video}")
    """
    try:
        # Auto-detect project ID if not provided
        if project_id is None:
            try:
                _, project_id = default()
                if not project_id:
                    # Try to get from environment
                    project_id = os.environ.get('GOOGLE_CLOUD_PROJECT')
                    if not project_id:
                        raise VEO2VideoGenerationError(
                            "Project ID could not be auto-detected. Please provide project_id parameter."
                        )
            except Exception as e:
                raise VEO2VideoGenerationError(f"Failed to auto-detect project ID: {str(e)}")

        # Initialize pipeline
        pipeline = VEO2VideoPipeline(project_id=project_id)

        # Execute complete video generation pipeline
        final_video_path = pipeline.generate_sequential_videos(
            reference_image_path=reference_image_path,
            scene_prompts=scene_prompts,
            api_key=api_key,
            output_folder_name=output_folder_name
        )

        return final_video_path

    except Exception as e:
        logger.error(f"VEO-2 video pipeline creation failed: {str(e)}")
        raise


# Example usage and testing function
if __name__ == "__main__":
    """
    Example usage of the VEO-2 video generation pipeline.
    """

    # Example parameters
    example_params = {
        "reference_image_path": "/path/to/your/reference_image.jpg",
        "scene_prompts": [
            "A majestic eagle perched on a rocky cliff overlooking a vast canyon",
            "The eagle spreads its wings and takes flight into the golden sunset",
            "Soaring high above snow-capped mountains with clouds below",
            "The eagle gracefully lands near a crystal clear alpine lake"
        ],
        "api_key": "your-google-vertex-ai-api-key",
        "output_folder_name": "eagle_flight_sequence"
    }

    try:
        # Execute the pipeline
        final_video = create_veo2_video_pipeline(**example_params)
        print(f"✅ Video generation completed successfully!")
        print(f"📹 Final video saved to: {final_video}")

    except Exception as e:
        print(f"❌ Video generation failed: {str(e)}")
        sys.exit(1)


## Draft 2: Edited Using Gemini 2.5 Pro 06-05



In [ ]:
import os
import sys
import time
import json
import base64
import logging
import shutil
import subprocess
import io
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple

# Third-party imports, grouped for clarity
import cv2
import numpy as np
from PIL import Image
import requests
from google.auth import default, exceptions as auth_exceptions
from google.auth.transport.requests import Request
from google.cloud import storage
from google.api_core import exceptions as api_core_exceptions
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

# Configure professional-grade logging for clear debugging and monitoring
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


class VEOVideoGenerationError(Exception):
    """Custom exception for specific errors within the VEO video generation pipeline."""
    pass


# Define a set of HTTP status codes that are safe to retry on.
# These typically represent transient server issues or rate limiting.
RETRYABLE_STATUS_CODES = {429, 500, 502, 503, 504}

def _is_retryable_http_error(exception: BaseException) -> bool:
    """
    Custom tenacity retry condition to check for specific HTTP status codes.

    Args:
        exception (BaseException): The exception caught by tenacity.

    Returns:
        bool: True if the exception is an HTTPError with a retryable status code, False otherwise.
    """
    # Check if the exception is an instance of requests.exceptions.HTTPError
    return (
        isinstance(exception, requests.exceptions.HTTPError) and
        # Ensure the response attribute exists
        hasattr(exception, 'response') and
        # Check if the status code is in our set of retryable codes
        exception.response.status_code in RETRYABLE_STATUS_CODES
    )

# Define a reusable, robust retry strategy for API calls.
# This handles transient network errors and specific, retryable HTTP status codes.
retry_on_api_error = retry(
    # Use exponential backoff for waiting, starting at 4s, with a max of 60s between retries.
    wait=wait_exponential(multiplier=1, min=4, max=60),
    # Stop retrying after 10 attempts to prevent infinite loops.
    stop=stop_after_attempt(10),
    # Define the conditions for retrying: connection errors, timeouts, or specific HTTP errors.
    retry=retry_if_exception_type((
        requests.exceptions.ConnectionError,
        requests.exceptions.Timeout,
    )) | _is_retryable_http_error,
    # Log a warning before each retry attempt for better visibility.
    before_sleep=lambda retry_state: logger.warning(
        f"Retrying API call due to {retry_state.outcome.exception()}. "
        f"Attempt #{retry_state.attempt_number}, waiting {retry_state.next_action.sleep:.2f}s..."
    )
)


class VEOVideoPipeline:
    """
    A professional, implementation-grade class for Google's VEO video generation.

    This class orchestrates the complete workflow of generating sequential videos using
    Google's VEO model via its synchronous API. It handles authentication, request
    building, response parsing (for both base64 and GCS URI), frame extraction,
    and memory-efficient video stitching with ffmpeg.
    """

    # --- Constants based on the official VEO API documentation ---
    # The required OAuth scope for Vertex AI API access.
    AUTH_SCOPE = ["https://www.googleapis.com/auth/cloud-platform"]
    # The official, correct model identifier for VEO.
    MODEL_ID = "video-generation-001"
    # The standard Vertex AI method for synchronous predictions.
    API_METHOD = "predict"
    # A generous timeout for the synchronous API call to allow for video generation.
    API_TIMEOUT_SECONDS = 900

    def __init__(self, project_id: Optional[str] = None, location: str = "us-central1"):
        """
        Initializes the VEO video generation pipeline.

        Args:
            project_id (Optional[str]): Your Google Cloud project ID. If None, it will be
                                        auto-detected from the environment.
            location (str): The Google Cloud region for Vertex AI. Defaults to us-central1.

        Raises:
            VEOVideoGenerationError: If initialization or authentication fails.
        """
        # Validate that the location parameter is a non-empty string.
        if not isinstance(location, str) or not location:
            raise ValueError("location must be a non-empty string")

        # Store the project ID and location.
        self.project_id: Optional[str] = project_id
        self.location: str = location
        # Construct the base API endpoint URL.
        self.api_endpoint: str = f"https://{location}-aiplatform.googleapis.com"

        # Perform authentication upon initialization.
        self._initialize_authentication()

        # After attempting auto-detection, confirm that a project_id is set.
        if not self.project_id:
            raise VEOVideoGenerationError(
                "Google Cloud project_id could not be determined. "
                "Please provide it explicitly or configure the environment."
            )

        # Log successful initialization.
        logger.info(f"VEOVideoPipeline initialized for project: {self.project_id}")

    def _initialize_authentication(self) -> None:
        """
        Initializes Google Cloud authentication using Application Default Credentials (ADC).

        This method explicitly requests the necessary scope for Vertex AI, ensuring
        the credentials obtained are valid for making API calls.

        Raises:
            VEOVideoGenerationError: If authentication fails.
        """
        try:
            # Use google.auth.default to find credentials in the environment, requesting the correct scope.
            self.credentials, detected_project_id = default(scopes=self.AUTH_SCOPE)

            # If the project_id was not provided during initialization, use the one detected by ADC.
            if self.project_id is None:
                self.project_id = detected_project_id

        except auth_exceptions.DefaultCredentialsError as e:
            # Provide a user-friendly error if credentials are not found.
            raise VEOVideoGenerationError(
                "Failed to find default credentials. Please run "
                "'gcloud auth application-default login' or configure the environment."
            ) from e
        except Exception as e:
            # Catch any other unexpected authentication errors.
            raise VEOVideoGenerationError(f"An unexpected error occurred during authentication: {e}") from e

    def _validate_inputs(
        self,
        reference_image_path: str,
        scene_prompts: List[str],
        output_folder_name: str
    ) -> None:
        """Validates all user-provided inputs for the video generation pipeline."""
        # Validate the path to the reference image.
        if not isinstance(reference_image_path, str) or not reference_image_path:
            raise ValueError("reference_image_path must be a non-empty string.")
        if not os.path.exists(reference_image_path):
            raise FileNotFoundError(f"Reference image file not found: {reference_image_path}")

        # Validate the list of scene prompts.
        if not isinstance(scene_prompts, list) or not scene_prompts:
            raise ValueError("scene_prompts must be a non-empty list of strings.")
        if not all(isinstance(prompt, str) and prompt.strip() for prompt in scene_prompts):
            raise ValueError("All items in scene_prompts must be non-empty strings.")

        # Validate the name for the output folder.
        if not isinstance(output_folder_name, str) or not output_folder_name:
            raise ValueError("output_folder_name must be a non-empty string.")

    def _create_output_directory(self, folder_name: str) -> Path:
        """Creates a subdirectory in the user's home directory for storing outputs."""
        try:
            # Use Path.home() for robust, cross-platform path resolution.
            home_dir = Path.home()
            # Define the full path for the output directory.
            output_dir = home_dir / folder_name
            # Create the directory, including parent directories if needed, and set permissions.
            output_dir.mkdir(mode=0o755, parents=True, exist_ok=True)
            logger.info(f"Output directory ensured at: {output_dir}")
            return output_dir
        except OSError as e:
            raise VEOVideoGenerationError(f"Failed to create output directory '{folder_name}': {e}") from e

    def _load_and_encode_image(self, image_path: str) -> str:
        """Loads an image, resizes it for VEO, and encodes it to a base64 string."""
        try:
            # Open the image file using a context manager to ensure it's properly closed.
            with Image.open(image_path) as img:
                # Convert the image to RGB to remove any alpha channel (e.g., from PNGs).
                if img.mode != 'RGB':
                    img = img.convert('RGB')

                # Resize the image to VEO's recommended dimensions while preserving aspect ratio.
                img = self._resize_image_for_veo(img)

                # Use an in-memory buffer to avoid writing a temporary file to disk.
                buffer = io.BytesIO()

                # Save the processed image to the buffer in JPEG format for efficiency.
                img.save(buffer, format='JPEG', quality=95)

                # Get the byte value from the buffer.
                image_bytes = buffer.getvalue()

                # Encode the image bytes to a base64 string and decode to utf-8.
                base64_encoded = base64.b64encode(image_bytes).decode('utf-8')

                logger.info(f"Successfully loaded and encoded image: {image_path}")
                return base64_encoded

        except FileNotFoundError:
            raise VEOVideoGenerationError(f"Image file not found at path: {image_path}")
        except Exception as e:
            raise VEOVideoGenerationError(f"Failed to load and encode image '{image_path}': {e}") from e

    def _resize_image_for_veo(self, img: Image.Image) -> Image.Image:
        """Resizes a PIL Image to VEO recommended dimensions, preserving aspect ratio."""
        # Get the current width and height of the image.
        width, height = img.size
        # Calculate the aspect ratio to determine orientation.
        aspect_ratio = width / height

        # Set target dimensions based on whether the image is landscape or portrait.
        target_width, target_height = (1024, 576) if aspect_ratio >= 1 else (576, 1024)

        # Resize the image using Lanczos resampling for high-quality results.
        resized_img = img.resize((target_width, target_height), Image.Resampling.LANCZOS)

        return resized_img

    def _build_request_body(self, prompt: str, image_data: Optional[str]) -> Dict[str, Any]:
        """
        Constructs the request body according to the official VEO API specification.

        Args:
            prompt (str): The text prompt for video generation.
            image_data (Optional[str]): Base64 encoded image data for video initialization.

        Returns:
            Dict[str, Any]: The structured request body for the API call.
        """
        # Define the instance payload, which contains the prompt.
        instance_payload = {"prompt": prompt}

        # If reference image data is provided, add it to the payload.
        if image_data:
            instance_payload["reference_image"] = {"image_bytes": image_data}

        # Construct the full request body with instances and parameters.
        request_body = {
            "instances": [instance_payload],
            "parameters": {
                "video_length": "4s",  # Specify desired video length (e.g., '4s', '15s').
                "fps": 24,             # Specify frames per second.
                "seed": np.random.randint(0, 2**32 - 1) # Use a random seed for creative variety.
            }
        }
        return request_body

    @retry_on_api_error
    def _generate_video_segment(self, prompt: str, image_data: Optional[str]) -> Dict[str, Any]:
        """
        Generates a single video segment using the synchronous VEO API.

        This function sends a request to the model and directly receives the
        generated video data or a GCS URI in the response.

        Args:
            prompt (str): The text prompt for the video segment.
            image_data (Optional[str]): Base64 encoded image data for initialization.

        Returns:
            Dict[str, Any]: The prediction dictionary from the API response, containing
                            either 'video_bytes' or 'gcs_uri'.

        Raises:
            VEOVideoGenerationError: If the API request fails after all retries.
        """
        try:
            # Refresh credentials before the API call to ensure the token is not expired.
            self.credentials.refresh(Request())

            # Construct the full API endpoint URL using the correct model and method.
            url = (f"{self.api_endpoint}/v1/projects/{self.project_id}/locations/{self.location}"
                   f"/publishers/google/models/{self.MODEL_ID}:{self.API_METHOD}")

            # Prepare the authorization and content-type headers.
            headers = {
                'Authorization': f'Bearer {self.credentials.token}',
                'Content-Type': 'application/json; charset=utf-8'
            }

            # Construct the request body using the dedicated helper method.
            request_body = self._build_request_body(prompt, image_data)

            logger.info("Sending request to VEO API. This may take several minutes...")
            # Send the POST request to the synchronous VEO API endpoint.
            response = requests.post(url, headers=headers, json=request_body, timeout=self.API_TIMEOUT_SECONDS)
            # Raise an exception for HTTP error codes (4xx or 5xx) to trigger tenacity retry if applicable.
            response.raise_for_status()

            # Parse the JSON response from the API.
            response_data = response.json()

            # Extract the list of predictions.
            predictions = response_data.get("predictions")
            if not predictions or not isinstance(predictions, list):
                raise VEOVideoGenerationError(f"API response is missing 'predictions'. Response: {response_data}")

            logger.info("Successfully received response from VEO API.")
            # Return the first prediction object.
            return predictions[0]

        except requests.exceptions.HTTPError as e:
            # Log the detailed error from the API response if available.
            logger.error(f"HTTP Error during video generation request: {e.response.text}")
            raise  # Re-raise to be handled by tenacity or the caller.
        except Exception as e:
            # Wrap other exceptions in our custom error type.
            raise VEOVideoGenerationError(f"Video generation failed: {e}") from e

    def _download_video_from_gcs(self, gcs_uri: str, local_path: Path) -> None:
        """Downloads a video from a Google Cloud Storage URI to a local path."""
        try:
            # Validate that the GCS URI has the correct prefix.
            if not gcs_uri.startswith('gs://'):
                raise ValueError(f"Invalid GCS URI format: {gcs_uri}")

            # Parse the bucket name and blob (object) name from the URI.
            bucket_name, blob_name = gcs_uri[5:].split('/', 1)

            # Initialize the Google Cloud Storage client with the correct project and credentials.
            storage_client = storage.Client(project=self.project_id, credentials=self.credentials)

            # Get the bucket and blob objects from the client.
            bucket = storage_client.bucket(bucket_name)
            blob = bucket.blob(blob_name)

            # Download the blob to the specified local file path.
            blob.download_to_filename(str(local_path))

            logger.info(f"Successfully downloaded video from {gcs_uri} to {local_path}")

        except (api_core_exceptions.NotFound, FileNotFoundError):
            raise VEOVideoGenerationError(f"GCS object not found at URI: {gcs_uri}")
        except Exception as e:
            raise VEOVideoGenerationError(f"Failed to download video from GCS: {e}") from e

    def _extract_last_frame(self, video_path: Path) -> np.ndarray:
        """Extracts the last frame from a video file using OpenCV."""
        # Initialize a video capture object with the path to the video file.
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            raise VEOVideoGenerationError(f"Cannot open video file for frame extraction: {video_path}")

        try:
            # Get the total number of frames in the video.
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            if total_frames == 0:
                raise VEOVideoGenerationError(f"Video appears to be empty (0 frames): {video_path}")

            # Set the capture position to the very last frame (which is at index total_frames - 1).
            cap.set(cv2.CAP_PROP_POS_FRAMES, total_frames - 1)

            # Read the frame at the current position.
            ret, frame = cap.read()

            # Check if the frame was read successfully.
            if not ret or frame is None:
                raise VEOVideoGenerationError(f"Failed to read the last frame from: {video_path}")

            logger.info(f"Successfully extracted last frame from video: {video_path}")
            return frame

        except Exception as e:
            raise VEOVideoGenerationError(f"Frame extraction failed for '{video_path}': {e}") from e
        finally:
            # Crucially, release the video capture object to free up resources.
            cap.release()

    def _frame_to_base64(self, frame: np.ndarray) -> str:
        """Converts an OpenCV frame (NumPy array) to a base64 encoded string."""
        try:
            # Convert the frame from OpenCV's BGR color space to PIL's RGB color space.
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Create a PIL Image object from the NumPy array.
            pil_image = Image.fromarray(frame_rgb)

            # Resize the frame to ensure it's compatible with VEO's input requirements.
            pil_image = self._resize_image_for_veo(pil_image)

            # Use an in-memory buffer to save the image without writing to disk.
            buffer = io.BytesIO()
            pil_image.save(buffer, format='JPEG', quality=95)

            # Get the byte value and encode it to a utf-8 decoded base64 string.
            image_bytes = buffer.getvalue()
            base64_encoded = base64.b64encode(image_bytes).decode('utf-8')

            return base64_encoded

        except Exception as e:
            raise VEOVideoGenerationError(f"Frame to base64 conversion failed: {e}") from e

    def _stitch_videos_ffmpeg(self, video_paths: List[Path], output_path: Path) -> None:
        """
        Stitches multiple video files into one using ffmpeg's concat demuxer.

        This method is highly memory-efficient as it avoids loading video clips
        into memory, making it suitable for stitching many or large files.

        Args:
            video_paths (List[Path]): An ordered list of video file paths to concatenate.
            output_path (Path): The path for the final stitched video file.

        Raises:
            VEOVideoGenerationError: If video stitching fails or ffmpeg is not found.
        """
        # Check if the ffmpeg executable is available in the system's PATH.
        if not shutil.which("ffmpeg"):
            raise FileNotFoundError(
                "ffmpeg not found. Please install ffmpeg and ensure it is in your system's PATH."
            )

        # Ensure there are videos to stitch.
        if not video_paths:
            raise VEOVideoGenerationError("No video clips were provided to stitch.")

        # Create a temporary manifest file for ffmpeg in the same directory as the output.
        manifest_path = output_path.with_suffix('.txt')

        try:
            # Write the list of video files to the manifest in the format required by ffmpeg.
            with open(manifest_path, 'w') as f:
                for video_path in video_paths:
                    if not video_path.exists():
                        raise FileNotFoundError(f"Video file for stitching not found: {video_path}")
                    # Use resolved absolute paths and quotes to handle special characters safely.
                    f.write(f"file '{video_path.resolve()}'\n")

            logger.info(f"Created ffmpeg manifest at: {manifest_path}")

            # Construct the ffmpeg command.
            # -y: Overwrite output file if it exists.
            # -f concat: Use the concat demuxer.
            # -safe 0: Allow absolute paths in the manifest file.
            # -i: Specify the input manifest file.
            # -c copy: Copy codecs without re-encoding for maximum speed and quality preservation.
            #          This assumes all segments have compatible codecs, which is true for VEO outputs.
            command = [
                "ffmpeg",
                "-y",
                "-f", "concat",
                "-safe", "0",
                "-i", str(manifest_path),
                "-c", "copy",
                str(output_path)
            ]

            logger.info(f"Executing ffmpeg command: {' '.join(command)}")

            # Execute the command, capturing stdout and stderr for debugging.
            result = subprocess.run(
                command,
                capture_output=True,
                text=True,
                check=False # We check the return code manually for better error reporting.
            )

            # Check if the ffmpeg command executed successfully.
            if result.returncode != 0:
                # If it failed, raise an error with the stderr from ffmpeg.
                raise VEOVideoGenerationError(
                    f"ffmpeg failed with exit code {result.returncode}.\n"
                    f"Stderr: {result.stderr}"
                )

            logger.info(f"Successfully stitched {len(video_paths)} videos into: {output_path}")

        except Exception as e:
            # Wrap any exception in our custom error type.
            raise VEOVideoGenerationError(f"Video stitching failed: {e}") from e
        finally:
            # Ensure the temporary manifest file is always cleaned up.
            if manifest_path.exists():
                manifest_path.unlink()
                logger.info(f"Cleaned up manifest file: {manifest_path}")

    def generate_sequential_videos(self,
        reference_image_path: str,
        scene_prompts: List[str],
        output_folder_name: str
    ) -> Path:
        """
        Executes the full pipeline to generate a sequence of chained videos.

        This is the main public method that orchestrates the entire process from
        the initial image to the final stitched video.

        Args:
            reference_image_path (str): Path to the initial reference image file.
            scene_prompts (List[str]): An ordered list of scene description prompts.
            output_folder_name (str): The name for the output folder in the home directory.

        Returns:
            Path: The path to the final, stitched video file.
        """
        try:
            # Step 0: Validate all inputs before starting any processing.
            self._validate_inputs(reference_image_path, scene_prompts, output_folder_name)

            # Step 1: Create the output directory to store all artifacts.
            output_dir = self._create_output_directory(output_folder_name)

            # Step 2: Load and encode the initial reference image to start the sequence.
            current_image_data = self._load_and_encode_image(reference_image_path)

            # Initialize a list to store the paths of the generated video segments.
            video_paths = []

            # Step 3: Iterate through each scene prompt to generate a video segment.
            for i, prompt in enumerate(scene_prompts):
                logger.info(f"--- Processing Scene {i+1}/{len(scene_prompts)} ---")
                logger.info(f"Prompt: '{prompt[:100]}...'")

                # Step 4: Generate a video segment using the current image and prompt.
                prediction = self._generate_video_segment(
                    prompt=prompt,
                    image_data=current_image_data
                )

                # Define a unique filename for this video segment.
                video_filename = f"segment_{i+1:03d}.mp4"
                video_path = output_dir / video_filename

                # Step 5: Process the API response, which could be bytes or a GCS URI.
                if 'video_bytes' in prediction:
                    # If video bytes are returned, decode them from base64 and save to a file.
                    logger.info(f"Received video as base64 bytes. Saving to {video_path}")
                    video_bytes = base64.b64decode(prediction['video_bytes'])
                    with open(video_path, 'wb') as f:
                        f.write(video_bytes)
                elif 'gcs_uri' in prediction:
                    # If a GCS URI is returned, download the video from the bucket.
                    logger.info(f"Received GCS URI. Downloading from {prediction['gcs_uri']}")
                    self._download_video_from_gcs(prediction['gcs_uri'], video_path)
                else:
                    # If neither is present, the response is invalid.
                    raise VEOVideoGenerationError(f"API prediction did not contain 'video_bytes' or 'gcs_uri': {prediction}")

                # Add the path of the newly created segment to our list.
                video_paths.append(video_path)

                # Step 6: For all but the last prompt, extract the last frame to seed the next segment.
                if i < len(scene_prompts) - 1:
                    logger.info("Extracting last frame to use as reference for the next segment...")
                    last_frame = self._extract_last_frame(video_path)
                    current_image_data = self._frame_to_base64(last_frame)

            # Step 7: Stitch all generated video segments into a single final video.
            final_video_path = output_dir / "final_stitched_video.mp4"
            self._stitch_videos_ffmpeg(video_paths, final_video_path)

            logger.info("--- Sequential video generation pipeline completed successfully. ---")
            logger.info(f"Final video saved to: {final_video_path}")

            return final_video_path

        except (VEOVideoGenerationError, ValueError, FileNotFoundError) as e:
            # Catch known, specific errors and log them before re-raising.
            logger.error(f"A predictable error occurred in the pipeline: {e}")
            raise
        except Exception as e:
            # Catch any other unexpected errors for robust failure handling.
            logger.error(f"An unexpected error occurred in the main pipeline: {e}", exc_info=True)
            raise VEOVideoGenerationError(f"An unexpected error occurred in the main pipeline: {e}") from e


def create_veo_video_pipeline(
    reference_image_path: str,
    scene_prompts: List[str],
    output_folder_name: str,
    project_id: Optional[str] = None
) -> Path:
    """
    A high-level factory function to create and run the VEO sequential video pipeline.

    This function abstracts away the class instantiation and execution, providing a
    simple, clean interface to generate a complete video from an image and prompts.

    Args:
        reference_image_path (str): Path to the initial reference image file.
        scene_prompts (List[str]): An ordered list of scene description prompts.
        output_folder_name (str): The name for the output folder in the home directory.
        project_id (Optional[str]): Your Google Cloud project ID. If None, it will be
                                    auto-detected from ADC or environment variables.

    Returns:
        Path: The path to the final, stitched video file.
    """
    try:
        # If project_id is not provided, attempt to get it from the environment as a fallback.
        if project_id is None:
            project_id = os.environ.get('GOOGLE_CLOUD_PROJECT')
            if project_id:
                logger.info(f"Using project ID from GOOGLE_CLOUD_PROJECT env var: {project_id}")

        # Initialize the VEO pipeline class.
        pipeline = VEOVideoPipeline(project_id=project_id)

        # Execute the complete video generation pipeline with the provided parameters.
        final_video_path = pipeline.generate_sequential_videos(
            reference_image_path=reference_image_path,
            scene_prompts=scene_prompts,
            output_folder_name=output_folder_name
        )

        return final_video_path

    except Exception as e:
        # Catch and log exceptions from the pipeline for clear top-level error reporting.
        logger.error(f"VEO video pipeline execution failed: {e}")
        # Re-raise the exception to allow the caller to handle it.
        raise


# Example usage and testing block
if __name__ == "__main__":
    """
    Example usage of the corrected and robust VEO video generation pipeline.

    *** IMPORTANT: CONFIGURATION REQUIRED ***
    1.  **AUTHENTICATION**: Run `gcloud auth application-default login` in your terminal.
    2.  **PROJECT ID**: Set the `GOOGLE_CLOUD_PROJECT` environment variable to your GCP project ID,
        or pass the `project_id` argument to `create_veo_video_pipeline`.
    3.  **APIs**: Ensure the Vertex AI API is enabled in your Google Cloud project.
    4.  **DEPENDENCIES**: Install required packages:
        pip install google-cloud-aiplatform google-cloud-storage opencv-python Pillow requests tenacity numpy
    5.  **FFMPEG**: Install ffmpeg (https://ffmpeg.org/download.html) and ensure it is in your system's PATH.
    6.  **REFERENCE IMAGE**: Replace the dummy image path with a path to your own image.
    """

    # --- CONFIGURE THESE PARAMETERS FOR YOUR RUN ---
    try:
        # Get the directory of the current script to create a sample image nearby.
        script_dir = Path(__file__).parent
    except NameError:
        # Fallback for interactive environments where __file__ is not defined.
        script_dir = Path.cwd()

    # Create a dummy reference image for testing if one doesn't exist.
    dummy_image_path = script_dir / "veo_reference_image.jpg"
    if not dummy_image_path.exists():
        print(f"Creating a dummy reference image at: {dummy_image_path}")
        # A blue 16:9 image.
        dummy_img = Image.new('RGB', (1024, 576), color='darkblue')
        dummy_img.save(dummy_image_path)

    # Define the parameters for the video generation task.
    example_params = {
        "reference_image_path": str(dummy_image_path),
        "scene_prompts": [
            "A majestic eagle perched on a rocky cliff overlooking a vast canyon, cinematic lighting.",
            "The eagle spreads its wings and takes flight into the golden sunset, slow motion.",
            "Soaring high above snow-capped mountains with clouds below, drone shot.",
            "The eagle gracefully lands near a crystal clear alpine lake, its reflection in the water."
        ],
        "output_folder_name": "veo_eagle_flight_sequence",
        # "project_id": "your-gcp-project-id"  # Optional: uncomment and set if needed.
    }

    print("--- Starting VEO Video Generation Pipeline ---")
    print(f"Reference Image: {example_params['reference_image_path']}")
    print(f"Output Folder Name: {example_params['output_folder_name']}")
    print(f"Number of Scenes: {len(example_params['scene_prompts'])}")

    try:
        # Execute the pipeline.
        start_time = time.time()
        final_video = create_veo_video_pipeline(**example_params)
        end_time = time.time()

        # Print a success message with the final output path.
        print("\n" + "="*60)
        print("✅ Video generation pipeline completed successfully!")
        print(f"📹 Final video saved to: {final_video}")
        print(f"⏱️ Total execution time: {end_time - start_time:.2f} seconds.")
        print("="*60)

    except (VEOVideoGenerationError, ValueError, FileNotFoundError) as e:
        # Handle known pipeline failures gracefully.
        print(f"\n❌ PIPELINE FAILED: {e}")
        sys.exit(1)
    except Exception as e:
        # Handle any other unexpected errors.
        print(f"\n❌ AN UNEXPECTED ERROR OCCURRED: {e}")
        sys.exit(1)


In [ ]:
# --- Standard Library Imports ---
import os
import sys
import time
import json
import base64
import logging
import shutil
import subprocess
import io
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple

# --- Third-party Imports, Grouped for Clarity ---
import cv2
import numpy as np
from numpy.typing import NDArray
from PIL import Image
import requests
from google.auth import default, exceptions as auth_exceptions
from google.auth.transport.requests import Request
from google.cloud import storage
from google.api_core import exceptions as api_core_exceptions
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

# --- Professional-Grade Logging Configuration ---
# Configure a logger for clear, standardized output for debugging and monitoring.
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
# Instantiate the logger using the standard __name__ convention.
logger = logging.getLogger(__name__)


# --- Centralized Configuration Class ---
class VEOConfig:
    """
    A centralized, immutable configuration class for VEO API constants.

    This provides a single source of truth for static parameters, improving
    maintainability and preventing the use of "magic" strings or numbers
    in the pipeline logic.
    """
    # The required OAuth scope for authenticating with the Vertex AI API.
    AUTH_SCOPE: List[str] = ["https://www.googleapis.com/auth/cloud-platform"]
    # The official, versioned model identifier for Google's VEO model.
    MODEL_ID: str = "video-generation-001"
    # The standard Vertex AI API method for synchronous (blocking) predictions.
    API_METHOD: str = "predict"
    # A generous timeout for the synchronous API call to allow for video generation.
    API_TIMEOUT_SECONDS: int = 900
    # The set of HTTP status codes that indicate transient server-side issues safe to retry.
    RETRYABLE_STATUS_CODES: set[int] = {429, 500, 502, 503, 504}
    # Recommended VEO dimensions for landscape videos (width, height).
    LANDSCAPE_DIMS: Tuple[int, int] = (1024, 576)
    # Recommended VEO dimensions for portrait videos (width, height).
    PORTRAIT_DIMS: Tuple[int, int] = (576, 1024)
    # Default video generation parameters.
    DEFAULT_VIDEO_LENGTH: str = "4s"
    DEFAULT_FPS: int = 24
    # JPEG quality for encoding reference frames. 95 is a high-quality setting.
    JPEG_ENCODING_QUALITY: int = 95


# --- Custom Exception for Pipeline-Specific Errors ---
class VEOVideoGenerationError(Exception):
    """Custom exception for specific, identifiable errors within the VEO video generation pipeline."""
    pass


# --- Robust Retry Logic for API Calls ---
def _is_retryable_http_error(exception: BaseException) -> bool:
    """
    Custom tenacity retry condition to check for specific HTTP status codes.

    This function determines if a caught exception is a requests.HTTPError with a
    status code that is defined as retryable in VEOConfig.

    Args:
        exception (BaseException): The exception caught by tenacity.

    Returns:
        bool: True if the exception is an HTTPError with a retryable status code, False otherwise.
    """
    # Return True only if the exception is an HTTPError and its status code is in our retryable set.
    return (
        isinstance(exception, requests.exceptions.HTTPError) and
        hasattr(exception, 'response') and
        exception.response.status_code in VEOConfig.RETRYABLE_STATUS_CODES
    )

# Define a reusable, robust retry strategy for API calls using the tenacity library.
retry_on_api_error = retry(
    # Use exponential backoff, starting at 4s and maxing out at 60s between retries.
    wait=wait_exponential(multiplier=1, min=4, max=60),
    # Stop retrying after 10 attempts to prevent indefinite loops and fail gracefully.
    stop=stop_after_attempt(10),
    # Define the conditions for retrying: network errors or specific HTTP server errors.
    retry=retry_if_exception_type((
        requests.exceptions.ConnectionError,
        requests.exceptions.Timeout,
    )) | _is_retryable_http_error,
    # Log a warning before each retry attempt for visibility into transient failures.
    before_sleep=lambda retry_state: logger.warning(
        f"Retrying API call due to {retry_state.outcome.exception()}. "
        f"Attempt #{retry_state.attempt_number}, waiting {retry_state.next_action.sleep:.2f}s..."
    )
)


class VEOVideoPipeline:
    """
    A professional, implementation-grade class for Google's VEO video generation.

    This class orchestrates the complete workflow of generating sequential videos using
    Google's VEO model via its synchronous API. It handles authentication, request
    building, response parsing, frame extraction, and memory-efficient video stitching.
    """

    def __init__(self, project_id: Optional[str] = None, location: str = "us-central1"):
        """
        Initializes the VEO video generation pipeline.

        Args:
            project_id (Optional[str]): Your Google Cloud project ID. If None, it will be
                                        auto-detected from the environment.
            location (str): The Google Cloud region for Vertex AI. Defaults to us-central1.

        Raises:
            VEOVideoGenerationError: If initialization or authentication fails.
            ValueError: If input parameters are invalid.
        """
        # Validate that the location parameter is a non-empty string.
        if not isinstance(location, str) or not location:
            raise ValueError("location must be a non-empty string")

        # Store the project ID and location as instance-specific attributes.
        self.project_id: Optional[str] = project_id
        self.location: str = location
        # Construct the base API endpoint URL for Vertex AI.
        self.api_endpoint: str = f"https://{location}-aiplatform.googleapis.com"

        # Perform authentication immediately upon initialization to fail fast.
        self._initialize_authentication()

        # After attempting auto-detection, confirm that a project_id is set.
        if not self.project_id:
            raise VEOVideoGenerationError(
                "Google Cloud project_id could not be determined. "
                "Please provide it explicitly or configure the environment."
            )

        # Log successful initialization for monitoring purposes.
        logger.info(f"VEOVideoPipeline initialized for project: {self.project_id}")

    def _initialize_authentication(self) -> None:
        """
        Initializes Google Cloud authentication using Application Default Credentials (ADC).

        This method explicitly requests the necessary scope for Vertex AI, ensuring
        the credentials obtained are valid for making API calls.

        Raises:
            VEOVideoGenerationError: If authentication fails for any reason.
        """
        try:
            # Use google.auth.default to find credentials, requesting the correct scope from VEOConfig.
            self.credentials, detected_project_id = default(scopes=VEOConfig.AUTH_SCOPE)

            # If the project_id was not provided during initialization, use the one detected by ADC.
            if self.project_id is None:
                self.project_id = detected_project_id

        except auth_exceptions.DefaultCredentialsError as e:
            # Provide a user-friendly error if credentials are not found in the environment.
            raise VEOVideoGenerationError(
                "Failed to find default credentials. Please run "
                "'gcloud auth application-default login' or configure the environment."
            ) from e
        except Exception as e:
            # Catch any other unexpected authentication errors and wrap them in our custom exception.
            raise VEOVideoGenerationError(f"An unexpected error occurred during authentication: {e}") from e

    def _validate_inputs(
        self,
        reference_image_path: str,
        scene_prompts: List[str],
        output_folder_name: str
    ) -> None:
        """Validates all user-provided inputs for the video generation pipeline."""
        # Validate the path to the reference image is a non-empty string.
        if not isinstance(reference_image_path, str) or not reference_image_path:
            raise ValueError("reference_image_path must be a non-empty string.")
        # Validate the reference image file actually exists.
        if not os.path.exists(reference_image_path):
            raise FileNotFoundError(f"Reference image file not found: {reference_image_path}")

        # Validate that scene_prompts is a non-empty list.
        if not isinstance(scene_prompts, list) or not scene_prompts:
            raise ValueError("scene_prompts must be a non-empty list of strings.")
        # Validate that every item in the list is a non-empty, non-whitespace string.
        if not all(isinstance(prompt, str) and prompt.strip() for prompt in scene_prompts):
            raise ValueError("All items in scene_prompts must be non-empty strings.")

        # Validate the name for the output folder is a non-empty string.
        if not isinstance(output_folder_name, str) or not output_folder_name:
            raise ValueError("output_folder_name must be a non-empty string.")

    def _create_output_directory(self, folder_name: str) -> Path:
        """Creates a subdirectory in the user's home directory for storing outputs."""
        try:
            # Use Path.home() for robust, cross-platform path resolution to the user's home directory.
            home_dir = Path.home()
            # Define the full path for the output directory.
            output_dir = home_dir / folder_name
            # Create the directory, including parent directories if needed, and set standard permissions.
            output_dir.mkdir(mode=0o755, parents=True, exist_ok=True)
            # Log the successful creation or confirmation of the directory.
            logger.info(f"Output directory ensured at: {output_dir}")
            return output_dir
        except OSError as e:
            # Raise a custom error if directory creation fails due to OS-level issues.
            raise VEOVideoGenerationError(f"Failed to create output directory '{folder_name}': {e}") from e

    def _load_and_encode_image(self, image_path: str) -> str:
        """Loads an image, resizes it for VEO, and encodes it to a base64 string."""
        try:
            # Open the image file using a context manager to ensure it's properly closed.
            with Image.open(image_path) as img:
                # Convert the image to RGB to remove any alpha channel (e.g., from PNGs), ensuring compatibility.
                if img.mode != 'RGB':
                    img = img.convert('RGB')

                # Resize the image to VEO's recommended dimensions while preserving aspect ratio.
                img = self._resize_image_for_veo(img)

                # Use an in-memory buffer to avoid writing a temporary file to disk, which is more efficient.
                buffer = io.BytesIO()

                # Save the processed image to the buffer in JPEG format for efficiency and smaller payload size.
                img.save(buffer, format='JPEG', quality=VEOConfig.JPEG_ENCODING_QUALITY)

                # Get the raw byte value from the buffer.
                image_bytes = buffer.getvalue()

                # Encode the image bytes to a base64 string and decode to utf-8 for JSON serialization.
                base64_encoded = base64.b64encode(image_bytes).decode('utf-8')

                # Log the successful operation.
                logger.info(f"Successfully loaded and encoded image: {image_path}")
                return base64_encoded

        except FileNotFoundError:
            # Raise a specific error if the file does not exist.
            raise VEOVideoGenerationError(f"Image file not found at path: {image_path}")
        except Exception as e:
            # Wrap any other image processing errors in our custom exception.
            raise VEOVideoGenerationError(f"Failed to load and encode image '{image_path}': {e}") from e

    def _resize_image_for_veo(self, img: Image.Image) -> Image.Image:
        """Resizes a PIL Image to VEO recommended dimensions, preserving aspect ratio."""
        # Get the current width and height of the image.
        width, height = img.size
        # Calculate the aspect ratio to determine if the image is landscape, portrait, or square.
        aspect_ratio = width / height

        # Set target dimensions from VEOConfig based on whether the image is landscape (or square) or portrait.
        target_dims = VEOConfig.LANDSCAPE_DIMS if aspect_ratio >= 1 else VEOConfig.PORTRAIT_DIMS

        # Resize the image using Lanczos resampling, which provides high-quality downscaling results.
        resized_img = img.resize(target_dims, Image.Resampling.LANCZOS)

        return resized_img

    def _build_request_body(self, prompt: str, image_data: Optional[str]) -> Dict[str, Any]:
        """Constructs the request body according to the official VEO API specification."""
        # Define the instance payload, which contains the core prompt.
        instance_payload = {"prompt": prompt}

        # If reference image data is provided (for the first or subsequent segments), add it to the payload.
        if image_data:
            instance_payload["reference_image"] = {"image_bytes": image_data}

        # Construct the full request body with instances and parameters from VEOConfig.
        request_body = {
            "instances": [instance_payload],
            "parameters": {
                "video_length": VEOConfig.DEFAULT_VIDEO_LENGTH,
                "fps": VEOConfig.DEFAULT_FPS,
                "seed": np.random.randint(0, 2**32 - 1) # Use a random seed for creative variety.
            }
        }
        return request_body

    @retry_on_api_error
    def _generate_video_segment(self, prompt: str, image_data: Optional[str]) -> Dict[str, Any]:
        """
        Generates a single video segment using the synchronous VEO API with robust retries.

        Args:
            prompt (str): The text prompt for the video segment.
            image_data (Optional[str]): Base64 encoded image data for initialization.

        Returns:
            Dict[str, Any]: The prediction dictionary from the API response.
        """
        try:
            # Refresh credentials before the API call to ensure the auth token is not expired.
            self.credentials.refresh(Request())

            # Construct the full API endpoint URL using constants from VEOConfig for accuracy.
            url = (f"{self.api_endpoint}/v1/projects/{self.project_id}/locations/{self.location}"
                   f"/publishers/google/models/{VEOConfig.MODEL_ID}:{VEOConfig.API_METHOD}")

            # Prepare the authorization and content-type headers for the request.
            headers = {
                'Authorization': f'Bearer {self.credentials.token}',
                'Content-Type': 'application/json; charset=utf-8'
            }

            # Construct the request body using the dedicated helper method.
            request_body = self._build_request_body(prompt, image_data)

            # Log the request initiation. This is a long-running operation.
            logger.info("Sending request to VEO API. This may take several minutes...")
            # Send the POST request to the VEO API with a configured timeout.
            response = requests.post(url, headers=headers, json=request_body, timeout=VEOConfig.API_TIMEOUT_SECONDS)
            # Raise an exception for HTTP error codes (4xx or 5xx), which tenacity will catch for retries.
            response.raise_for_status()

            # Parse the JSON response from the API.
            response_data = response.json()

            # Extract the list of predictions from the response body.
            predictions = response_data.get("predictions")
            # Validate that the predictions list exists and is not empty.
            if not predictions or not isinstance(predictions, list):
                raise VEOVideoGenerationError(f"API response is missing 'predictions'. Response: {response_data}")

            # Log the successful receipt of the response.
            logger.info("Successfully received response from VEO API.")
            # Return the first prediction object, as we send one instance at a time.
            return predictions[0]

        except requests.exceptions.HTTPError as e:
            # Log the detailed error from the API response body if available for easier debugging.
            logger.error(f"HTTP Error during video generation request: {e.response.text}")
            raise  # Re-raise the exception to be handled by the tenacity retry decorator or the caller.
        except Exception as e:
            # Wrap any other exceptions in our custom error type for consistent error handling.
            raise VEOVideoGenerationError(f"Video generation failed: {e}") from e

    def _download_video_from_gcs(self, gcs_uri: str, local_path: Path) -> None:
        """Downloads a video from a Google Cloud Storage URI to a local path."""
        try:
            # Validate that the GCS URI has the correct 'gs://' prefix.
            if not gcs_uri.startswith('gs://'):
                raise ValueError(f"Invalid GCS URI format: {gcs_uri}")

            # Parse the bucket name and blob (object) name from the URI string.
            bucket_name, blob_name = gcs_uri[5:].split('/', 1)

            # Initialize the Google Cloud Storage client with the correct project and credentials.
            storage_client = storage.Client(project=self.project_id, credentials=self.credentials)

            # Get a reference to the bucket and blob objects.
            bucket = storage_client.bucket(bucket_name)
            blob = bucket.blob(blob_name)

            # Download the blob's contents to the specified local file path.
            blob.download_to_filename(str(local_path))

            # Log the successful download.
            logger.info(f"Successfully downloaded video from {gcs_uri} to {local_path}")

        except (api_core_exceptions.NotFound, FileNotFoundError):
            # Handle cases where the GCS object does not exist.
            raise VEOVideoGenerationError(f"GCS object not found at URI: {gcs_uri}")
        except Exception as e:
            # Wrap any other GCS-related errors.
            raise VEOVideoGenerationError(f"Failed to download video from GCS: {e}") from e

    def _extract_last_frame(self, video_path: Path) -> NDArray[np.uint8]:
        """Extracts the last frame from a video file using OpenCV."""
        # Initialize a video capture object with the path to the video file.
        cap = cv2.VideoCapture(str(video_path))
        # Check if the video file was opened successfully.
        if not cap.isOpened():
            raise VEOVideoGenerationError(f"Cannot open video file for frame extraction: {video_path}")

        try:
            # Get the total number of frames in the video.
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            # Ensure the video is not empty.
            if total_frames == 0:
                raise VEOVideoGenerationError(f"Video appears to be empty (0 frames): {video_path}")

            # Set the capture position to the very last frame (index is total_frames - 1).
            cap.set(cv2.CAP_PROP_POS_FRAMES, total_frames - 1)

            # Read the frame at the new position.
            ret, frame = cap.read()

            # Check if the frame was read successfully. 'ret' will be False if it fails.
            if not ret or frame is None:
                raise VEOVideoGenerationError(f"Failed to read the last frame from: {video_path}")

            # Log the successful extraction.
            logger.info(f"Successfully extracted last frame from video: {video_path}")
            # Return the frame as a NumPy array.
            return frame

        except Exception as e:
            # Wrap any OpenCV or other errors in our custom exception.
            raise VEOVideoGenerationError(f"Frame extraction failed for '{video_path}': {e}") from e
        finally:
            # Crucially, release the video capture object to free up system resources.
            cap.release()

    def _frame_to_base64(self, frame: NDArray[np.uint8]) -> str:
        """Converts an OpenCV frame (NumPy array) to a base64 encoded string."""
        try:
            # Convert the frame from OpenCV's default BGR color space to PIL's expected RGB color space.
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Create a PIL Image object from the NumPy array for easier processing.
            pil_image = Image.fromarray(frame_rgb)

            # Resize the frame to ensure it's compatible with VEO's input requirements, using the same logic.
            pil_image = self._resize_image_for_veo(pil_image)

            # Use an in-memory buffer to save the image without writing to disk.
            buffer = io.BytesIO()
            # Save the image to the buffer as a high-quality JPEG.
            pil_image.save(buffer, format='JPEG', quality=VEOConfig.JPEG_ENCODING_QUALITY)

            # Get the raw byte value and encode it to a utf-8 decoded base64 string.
            image_bytes = buffer.getvalue()
            base64_encoded = base64.b64encode(image_bytes).decode('utf-8')

            return base64_encoded

        except Exception as e:
            # Wrap any conversion errors.
            raise VEOVideoGenerationError(f"Frame to base64 conversion failed: {e}") from e

    def _stitch_videos_ffmpeg(self, video_paths: List[Path], output_path: Path) -> None:
        """Stitches multiple video files into one using ffmpeg's concat demuxer for efficiency."""
        # Check if the ffmpeg executable is available in the system's PATH before proceeding.
        if not shutil.which("ffmpeg"):
            raise FileNotFoundError(
                "ffmpeg not found. Please install ffmpeg and ensure it is in your system's PATH."
            )

        # Ensure there is at least one video to process.
        if not video_paths:
            raise VEOVideoGenerationError("No video clips were provided to stitch.")

        # Create a temporary manifest file for ffmpeg in the same directory as the output.
        manifest_path = output_path.with_suffix('.txt')

        try:
            # Write the list of video files to the manifest in the format required by ffmpeg's concat demuxer.
            with open(manifest_path, 'w') as f:
                for video_path in video_paths:
                    # Verify each video file exists before adding it to the manifest.
                    if not video_path.exists():
                        raise FileNotFoundError(f"Video file for stitching not found: {video_path}")
                    # Use resolved absolute paths and quotes to handle spaces or special characters safely.
                    f.write(f"file '{video_path.resolve()}'\n")

            # Log the creation of the manifest file.
            logger.info(f"Created ffmpeg manifest at: {manifest_path}")

            # Construct the ffmpeg command for efficient, lossless concatenation.
            command = [
                "ffmpeg",
                "-y",                   # Overwrite output file if it exists.
                "-f", "concat",         # Use the concat demuxer.
                "-safe", "0",           # Allow absolute paths in the manifest file (required for resolved paths).
                "-i", str(manifest_path), # Specify the input manifest file.
                "-c", "copy",           # Copy codecs without re-encoding for maximum speed and quality preservation.
                str(output_path)        # Specify the final output file path.
            ]

            # Log the command being executed for debugging purposes.
            logger.info(f"Executing ffmpeg command: {' '.join(command)}")

            # Execute the command, capturing stdout and stderr for detailed error reporting.
            result = subprocess.run(
                command,
                capture_output=True,
                text=True,
                check=False # We check the return code manually for better error logging.
            )

            # Check if the ffmpeg command executed successfully.
            if result.returncode != 0:
                # If it failed, raise an error with the detailed stderr from ffmpeg.
                raise VEOVideoGenerationError(
                    f"ffmpeg failed with exit code {result.returncode}.\n"
                    f"Stderr: {result.stderr}"
                )

            # Log the successful completion of the stitching process.
            logger.info(f"Successfully stitched {len(video_paths)} videos into: {output_path}")

        except Exception as e:
            # Wrap any exception in our custom error type.
            raise VEOVideoGenerationError(f"Video stitching failed: {e}") from e
        finally:
            # Ensure the temporary manifest file is always cleaned up, even if errors occur.
            if manifest_path.exists():
                manifest_path.unlink()
                logger.info(f"Cleaned up manifest file: {manifest_path}")

    def generate_sequential_videos(self,
        reference_image_path: str,
        scene_prompts: List[str],
        output_folder_name: str
    ) -> Path:
        """Executes the full pipeline to generate a sequence of chained videos."""
        try:
            # Step 0: Validate all inputs before starting any expensive processing.
            self._validate_inputs(reference_image_path, scene_prompts, output_folder_name)

            # Step 1: Create the output directory to store all generated artifacts.
            output_dir = self._create_output_directory(output_folder_name)

            # Step 2: Load and encode the initial reference image to start the sequence.
            current_image_data: Optional[str] = self._load_and_encode_image(reference_image_path)

            # Initialize a list to store the paths of the generated video segments for later stitching.
            video_paths: List[Path] = []

            # Step 3: Iterate through each scene prompt to generate a corresponding video segment.
            for i, prompt in enumerate(scene_prompts):
                logger.info(f"--- Processing Scene {i+1}/{len(scene_prompts)} ---")
                logger.info(f"Prompt: '{prompt[:100]}...'")

                # Step 4: Generate a video segment using the current image data and prompt.
                prediction = self._generate_video_segment(
                    prompt=prompt,
                    image_data=current_image_data
                )

                # Define a unique, ordered filename for this video segment.
                video_filename = f"segment_{i+1:03d}.mp4"
                video_path = output_dir / video_filename

                # Step 5: Process the API response, which could contain bytes or a GCS URI.
                if 'video_bytes' in prediction:
                    # If video bytes are returned directly, decode them from base64 and save to a file.
                    logger.info(f"Received video as base64 bytes. Saving to {video_path}")
                    video_bytes = base64.b64decode(prediction['video_bytes'])
                    with open(video_path, 'wb') as f:
                        f.write(video_bytes)
                elif 'gcs_uri' in prediction:
                    # If a GCS URI is returned, download the video from the bucket.
                    logger.info(f"Received GCS URI. Downloading from {prediction['gcs_uri']}")
                    self._download_video_from_gcs(prediction['gcs_uri'], video_path)
                else:
                    # If neither key is present, the API response is invalid.
                    raise VEOVideoGenerationError(f"API prediction did not contain 'video_bytes' or 'gcs_uri': {prediction}")

                # Add the path of the newly created segment to our list for the final stitching step.
                video_paths.append(video_path)

                # Step 6: For all but the last prompt, extract the last frame to seed the next segment.
                if i < len(scene_prompts) - 1:
                    logger.info("Extracting last frame to use as reference for the next segment...")
                    last_frame = self._extract_last_frame(video_path)
                    current_image_data = self._frame_to_base64(last_frame)

            # Step 7: Stitch all generated video segments into a single final video file.
            final_video_path = output_dir / "final_stitched_video.mp4"
            self._stitch_videos_ffmpeg(video_paths, final_video_path)

            # Log the successful completion of the entire pipeline.
            logger.info("--- Sequential video generation pipeline completed successfully. ---")
            logger.info(f"Final video saved to: {final_video_path}")

            # Return the path to the final product.
            return final_video_path

        except (VEOVideoGenerationError, ValueError, FileNotFoundError) as e:
            # Catch known, specific errors and log them clearly before re-raising.
            logger.error(f"A predictable error occurred in the pipeline: {e}")
            raise
        except Exception as e:
            # Catch any other unexpected errors for robust failure handling.
            logger.error(f"An unexpected error occurred in the main pipeline: {e}", exc_info=True)
            raise VEOVideoGenerationError(f"An unexpected error occurred in the main pipeline: {e}") from e


def create_veo_video_pipeline(
    reference_image_path: str,
    scene_prompts: List[str],
    output_folder_name: str,
    project_id: Optional[str] = None
) -> Path:
    """
    A high-level factory function to create and run the VEO sequential video pipeline.

    This function abstracts away the class instantiation and execution, providing a
    simple, clean interface to generate a complete video from an image and prompts.

    Args:
        reference_image_path (str): Path to the initial reference image file.
        scene_prompts (List[str]): An ordered list of scene description prompts.
        output_folder_name (str): The name for the output folder in the home directory.
        project_id (Optional[str]): Your Google Cloud project ID. If None, it will be
                                    auto-detected from ADC or environment variables.

    Returns:
        Path: The path to the final, stitched video file.
    """
    try:
        # If project_id is not provided, attempt to get it from the environment as a fallback.
        if project_id is None:
            project_id = os.environ.get('GOOGLE_CLOUD_PROJECT')
            if project_id:
                logger.info(f"Using project ID from GOOGLE_CLOUD_PROJECT env var: {project_id}")

        # Initialize the VEO pipeline class with the determined project ID.
        pipeline = VEOVideoPipeline(project_id=project_id)

        # Execute the complete video generation pipeline with the provided parameters.
        final_video_path = pipeline.generate_sequential_videos(
            reference_image_path=reference_image_path,
            scene_prompts=scene_prompts,
            output_folder_name=output_folder_name
        )

        return final_video_path

    except Exception as e:
        # Catch and log any exceptions from the pipeline for clear top-level error reporting.
        logger.error(f"VEO video pipeline execution failed: {e}")
        # Re-raise the exception to allow the calling script to handle it as needed.
        raise


# --- Example Usage and Testing Block ---
if __name__ == "__main__":
    """
    Example usage of the corrected and robust VEO video generation pipeline.

    *** IMPORTANT: CONFIGURATION REQUIRED ***
    1.  **AUTHENTICATION**: Run `gcloud auth application-default login` in your terminal.
    2.  **PROJECT ID**: Set the `GOOGLE_CLOUD_PROJECT` environment variable to your GCP project ID,
        or pass the `project_id` argument to `create_veo_video_pipeline`.
    3.  **APIs**: Ensure the Vertex AI API is enabled in your Google Cloud project.
    4.  **DEPENDENCIES**: Install required packages:
        pip install google-cloud-aiplatform google-cloud-storage opencv-python Pillow requests tenacity numpy
    5.  **FFMPEG**: Install ffmpeg (https://ffmpeg.org/download.html) and ensure it is in your system's PATH.
    6.  **REFERENCE IMAGE**: Replace the dummy image path with a path to your own image.
    """

    # --- CONFIGURE THESE PARAMETERS FOR YOUR RUN ---
    try:
        # Get the directory of the current script to create a sample image nearby.
        script_dir = Path(__file__).parent
    except NameError:
        # Fallback for interactive environments (like Jupyter) where __file__ is not defined.
        script_dir = Path.cwd()

    # Create a dummy reference image for testing if one doesn't exist.
    dummy_image_path = script_dir / "veo_reference_image.jpg"
    if not dummy_image_path.exists():
        print(f"Creating a dummy reference image at: {dummy_image_path}")
        # A blue 16:9 image matching VEO's landscape dimensions.
        dummy_img = Image.new('RGB', VEOConfig.LANDSCAPE_DIMS, color='darkblue')
        dummy_img.save(dummy_image_path)

    # Define the parameters for the video generation task.
    example_params = {
        "reference_image_path": str(dummy_image_path),
        "scene_prompts": [
            "A majestic eagle perched on a rocky cliff overlooking a vast canyon, cinematic lighting.",
            "The eagle spreads its wings and takes flight into the golden sunset, slow motion.",
            "Soaring high above snow-capped mountains with clouds below, drone shot.",
            "The eagle gracefully lands near a crystal clear alpine lake, its reflection in the water."
        ],
        "output_folder_name": "veo_eagle_flight_sequence",
        # "project_id": "your-gcp-project-id"  # Optional: uncomment and set if needed.
    }

    print("--- Starting VEO Video Generation Pipeline ---")
    print(f"Reference Image: {example_params['reference_image_path']}")
    print(f"Output Folder Name: {example_params['output_folder_name']}")
    print(f"Number of Scenes: {len(example_params['scene_prompts'])}")

    try:
        # Execute the pipeline and time its execution.
        start_time = time.time()
        final_video = create_veo_video_pipeline(**example_params)
        end_time = time.time()

        # Print a clear success message with the final output path and total time.
        print("\n" + "="*60)
        print("✅ Video generation pipeline completed successfully!")
        print(f"📹 Final video saved to: {final_video}")
        print(f"⏱️ Total execution time: {end_time - start_time:.2f} seconds.")
        print("="*60)

    except (VEOVideoGenerationError, ValueError, FileNotFoundError) as e:
        # Handle known, controlled pipeline failures gracefully with a clear error message.
        print(f"\n❌ PIPELINE FAILED: {e}")
        sys.exit(1)
    except Exception as e:
        # Handle any other unexpected errors to prevent unhandled stack traces.
        print(f"\n❌ AN UNEXPECTED ERROR OCCURRED: {e}")
        sys.exit(1)
